In [ ]:
import random
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import numpy as np
import math
import random
import unidecode
import datetime
from time import strptime
import math
from IPython.core.display import display, HTML
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)
display(HTML("<style>.container { width:100% !important; }</style>"))

# Below are dicts, lists, and strings that will be used in later functions
header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
acronym_to_city_dict = {'ATL': 'Atlanta',
                        'WSH': 'Washington',
                        'NYM': 'New York Mets',
                        'PHI': 'Philadelphia',
                        'MIA': 'Miami',
                        'STL': 'St. Louis',
                        'MIL': 'Milwaukee',
                        'CHC': 'Chicago Cubs',
                        'CIN': 'Cincinatti',
                        'PIT': 'Pittsburg',
                        'LAD': 'Los Angeles Dodgers',
                        'ARI': 'Arizona',
                        'SF': 'San Francisco',
                        'COL': 'Colorado',
                        'SD': 'San Diego',
                        'NYY': 'New York Yankees',
                        'TB': 'Tampa Bay',
                        'BOS': 'Boston',
                        'TOR': 'Toronto',
                        'BAL': 'Baltimore',
                        'MIN': 'Minnesota',
                        'CLE': 'Cleveland',
                        'CHW': 'Chicago White Sox',
                        'KC': 'Kansas City',
                        'DET': 'Detroit',
                        'HOU': 'Houston',
                        'OAK': 'Oakland',
                        'TEX': 'Texas',
                        'LAA': 'Los Angeles Angels',
                        'SEA': 'Seattle'}
#acronym_to_city_dict = {'NYY': 'New York Yankees', 'ATL': 'Atlanta'}
nickname_to_city_dict = {'braves': 'Atlanta',
                        'nationals': 'Washington',
                        'mets': 'New York Mets',
                        'phillies': 'Philadelphia',
                        'marlins': 'Miami',
                        'cardinals': 'St. Louis',
                        'brewers': 'Milwaukee',
                        'cubs': 'Chicago Cubs',
                        'reds': 'Cincinatti',
                        'pirates': 'Pittsburg',
                        'dodgers': 'Los Angeles Dodgers',
                        'diamondbacks': 'Arizona',
                        'd-backs': 'Arizona',
                        'giants': 'San Francisco',
                        'rockies': 'Colorado',
                        'padres': 'San Diego',
                        'yankees': 'New York Yankees',
                        'rays': 'Tampa Bay',
                        'red sox': 'Boston',
                        'blue jays': 'Toronto',
                        'orioles': 'Baltimore',
                        'twins': 'Minnesota',
                        'indians': 'Cleveland',
                        'white sox': 'Chicago White Sox',
                        'royals': 'Kansas City',
                        'tigers': 'Detroit',
                        'astros': 'Houston',
                        'athletics': 'Oakland',
                        'rangers': 'Texas',
                        'angels': 'Los Angeles Angels',
                        'mariners': 'Seattle'}
league = {'Atlanta': 'NL',
          'Washington': 'NL',
          'New York Mets': 'NL',
          'Philadelphia': 'NL',
          'Miami': 'NL',
          'St. Louis': 'NL',
          'Milwaukee': 'NL',
          'Chicago Cubs': 'NL',
          'Cincinatti': 'NL',
          'Pittsburg': 'NL',
          'Los Angeles Dodgers': 'NL',
          'Arizona': 'NL',
          'San Francisco': 'NL',
          'Colorado': 'NL',
          'San Diego': 'NL',
          'New York Yankees': 'AL',
          'Tampa Bay': 'AL',
          'Boston': 'AL',
          'Toronto': 'AL',
          'Baltimore': 'AL',
          'Minnesota': 'AL',
          'Cleveland': 'AL',
          'Chicago White Sox': 'AL',
          'Kansas City': 'AL',
          'Detroit': 'AL',
          'Houston': 'AL',
          'Oakland': 'AL',
          'Texas': 'AL',
          'Los Angeles Angels': 'AL',
          'Seattle': 'AL'}
mlb_teams = ['braves', 'nationals', 'mets', 'phillies', 'marlins', 
         'cardinals', 'brewers', 'cubs', 'reds', 'pirates', 
         'dodgers', 'd-backs', 'giants', 'rockies', 'padres', 
         'yankees', 'rays', 'red sox', 'blue jays', 'orioles', 'twins',
         'indians', 'white sox', 'royals', 'tigers', 'astros', 'athletics',
         'rangers', 'angels', 'mariners']
new_player_id = {'Anthony Bemboom': '13854',
 'Jared Walsh': '18607',
 'Elliot Soto': 'sa390666',
 'Michael Hermosillo': '16285',
 'Taylor Ward': '17548',
 'Jo Adell': 'sa3004837',
 'Austin Wynns': '15271',
 'Ryan Mountcastle': 'sa874734',
 'Andrew Velazquez': '14196',
 'Ramon Urias': 'sa597169',
 'Rylan Bannon': 'sa3004162',
 'Yusniel Diaz': 'sa912848',
 'Ryan McKenna': 'sa875055',
 'Mason Williams': '11859',
 'Juan Centeno\xa0\xa0': '7168',
 'Bobby Dalbec': 'sa857701',
 'Josh Ockimey': 'sa829375',
 'Jonathan Arauz': 'sa877498',
 'C.J. Chatham': 'sa873984',
 'Dustin Pedroia\xa0\xa0': '8370',
 'Tzu-Wei Lin': '14678',
 'Seby Zavala': '18887',
 'Nick Madrigal': 'sa3008139',
 'Danny Mendick': '18889',
 'Nicky Delmonico': '13157',
 'Daniel Palka': '14897',
 'Luis Robert': 'sa3003396',
 'Luis Alexander Basabe': 'sa736915',
 'Beau Taylor': '12384',
 'Bobby Bradley': '17278',
 'Daniel Johnson': 'sa918018',
 'Bradley Zimmer': '16221',
 'Eric Haase': '14111',
 'Sergio Alcantara': 'sa739641',
 'Isaac Paredes': 'sa920275',
 'Jorge Bonifacio': '12174',
 'Daz Cameron': 'sa874736',
 'Troy Stokes Jr.': 'sa828871',
 'Salvador Perez': '7304',
 'Matt Reynolds': '13788',
 'Erick Mejia': '15817',
 'Nick Heath': 'sa918355',
 'Nick Gordon': 'sa828662',
 'Luke Raley': 'sa918048',
 'Brent Rooker': 'sa3003172',
 'Alex Kirilloff': 'sa917929',
 'Kyle Higashioka': '5517',
 'Erik Kratz': '4403',
 'Miguel Andujar': '15878',
 'Austin Allen': '18083',
 'Jonah Heim': 'sa737843',
 'Vimael Machin': 'sa602488',
 'Jorge Mateo': 'sa657910',
 'Dustin Fowler': '17098',
 'Buddy Reed': 'sa738683',
 'Skye Bolt': '17723',
 'Joseph Odom': 'sa738310',
 'Joe Hudson': '14412',
 'Evan White': 'sa915774',
 'Patrick Wisdom': '13602',
 'Sam Haggerty\xa0\xa0': '18054',
 'Donnie Walton': '19314',
 'Jake Fraley': '19260',
 'Eric Filia': 'sa827359',
 'Michael Perez': '12977',
 'Chris Herrmann': '9284',
 'Yoshitomo Tsutsugo': '27459',
 'Esteban Quiroz': 'sa596560',
 'Randy Arozarena': '19290',
 "Brian O'Grady": '16729',
 'Brendan McKay': '20186',
 'Greg Bird': '14131',
 'Eli White': 'sa738478',
 'Adolis Garcia': '19287',
 'Caleb Joseph': '7087',
 'Santiago Espinal': 'sa918188',
 'Anthony Alford': '14329',
 'Daulton Varsho': 'sa3002838',
 'Domingo Leyba\xa0\xa0': '16404',
 'Andy Young': 'sa919031',
 'Wyatt Mathisen': 'sa657937',
 'Alex Jackson': '17276',
 'Cristian Pache': 'sa916722',
 'Nico Hoerner': '21479',
 'Steven Souza Jr.': '5667',
 'Ian Miller': '15156',
 'Alex Blandino': '16271',
 'Shogo Akiyama': '27461',
 'Travis Jankowski\xa0\xa0': '13768',
 'Drew Butera': '3411',
 'Dom Nunez': '15055',
 'Chad Wallach': '17161',
 'Lewin Diaz': 'sa828420',
 'Christian Lopes': 'sa601044',
 'Magneuris Sierra': '17023',
 'Matt Kemp': '5631',
 'Monte Harrison': 'sa828691',
 'Jesus Sanchez': 'sa872787',
 'Garrett Stubbs': '18067',
 'Taylor Jones': 'sa876202',
 'Ronnie Dawson': 'sa873674',
 'Rocky Gale': '10533',
 'Gavin Lux': '19955',
 'Zach McKinstry': 'sa918920',
 'Edwin Rios': '18316',
 'David Freitas': '10619',
 'Jacob Nottingham': '16448',
 'Logan Morrison': '9205',
 'Mark Mathias': 'sa875006',
 'Corey Ray': 'sa738510',
 'Tyrone Taylor\xa0\xa0': '13675',
 'Raudy Read': '15766',
 'Tres Barrera': '19977',
 'Carter Kieboom': '19958',
 'Adrian Sanchez': '11338',
 'Andrew Stevenson': '17932',
 'Rene Rivera': '3648',
 'Jed Lowrie\xa0\xa0': '4418',
 'Luis Guillorme': '16451',
 'Yoenis Cespedes\xa0\xa0': '13110',
 'Deivy Grullon': '15988',
 'Arquimedes Gamboa': 'sa877497',
 'Alec Bohm': 'sa3007284',
 'Kyle Garlick': '18063',
 'Mickey Moniak': 'sa917920',
 'Kevin Kramer': '17689',
 "Ke'Bryan Hayes": 'sa874730',
 'Jason Martin': '16429',
 'Socrates Brito': '12944',
 'Andrew Knizner': '19514',
 'Rangel Ravelo': '10839',
 'Edmundo Sosa': '17022',
 'Max Schrock': 'sa658670',
 'Dylan Carlson': 'sa917940',
 'Justin Williams': '15490',
 'Lane Thomas': '16939',
 'Luis Torrens': '15905',
 'Jake Cronenworth': 'sa857694',
 'Abraham Almonte': '5486',
 'Franchy Cordero': '14567',
 'Edward Olivares': 'sa832015',
 'Rob Brantly': '10655',
 'Tyler Heineman': '13897',
 'Aramis Garcia\xa0\xa0': '16925',
 'Kean Wong': '15994',
 'Cristhian Adames': '6013',
 'Jaylin Davis': '19552',
 'Chris Shaw': '17738',
 'Drew Robinson': '11761',
 'Jose Siri': 'sa736940',
 'Joe McCarthy': 'sa858039'}
id_bats_dict = {'13854': 'L',
 'sa828871': 'R',
 '5667': 'R',
 '18607': 'L',
 'sa390666': 'R',
 '16285': 'R',
 '17548': 'R',
 'sa3004837': 'R',
 '15271': 'R',
 'sa874734': 'R',
 '14196': 'S',
 'sa597169': 'R',
 'sa3004162': 'R',
 'sa912848': 'R',
 'sa875055': 'R',
 '11859': 'L',
 '7168': 'L',
 'sa857701': 'R',
 'sa829375': 'L',
 'sa877498': 'S',
 'sa873984': 'R',
 '8370': 'R',
 '14678': 'L',
 '18887': 'R',
 'sa3008139': 'R',
 '18889': 'R',
 '13157': 'L',
 '14897': 'L',
 'sa3003396': 'R',
 'sa736915': 'S',
 '12384': 'L',
 '17278': 'L',
 'sa918018': 'L',
 '16221': 'L',
 '14111': 'R',
 'sa739641': 'S',
 'sa920275': 'R',
 '12174': 'R',
 'sa874736': 'R',
 '7304': 'R',
 '13788': 'R',
 '15817': 'S',
 'sa918355': 'L',
 'sa828662': 'L',
 'sa918048': 'L',
 'sa3003172': 'R',
 'sa917929': 'L',
 '5517': 'R',
 '4403': 'R',
 '15878': 'R',
 '18083': 'L',
 'sa737843': 'S',
 'sa602488': 'L',
 'sa657910': 'R',
 '17098': 'L',
 'sa738683': 'S',
 '17723': 'S',
 'sa738310': 'R',
 '14412': 'R',
 'sa915774': 'R',
 '13602': 'R',
 '18054': 'S',
 '19314': 'L',
 '19260': 'L',
 'sa827359': 'L',
 '12977': 'L',
 '9284': 'L',
 '27459': 'L',
 'sa596560': 'L',
 '19290': 'R',
 '16729': 'L',
 '20186': 'L',
 '14131': 'L',
 'sa738478': 'R',
 '19287': 'R',
 '7087': 'R',
 'sa918188': 'R',
 '14329': 'R',
 'sa3002838': 'L',
 '16404': 'S',
 'sa919031': 'R',
 'sa657937': 'R',
 '17276': 'R',
 'sa916722': 'R',
 '21479': 'R',
 '15156': 'L',
 '16271': 'R',
 '27461': 'L',
 '13768': 'L',
 '3411': 'R',
 '15055': 'L',
 '17161': 'R',
 'sa828420': 'L',
 'sa601044': 'R',
 '17023': 'L',
 '5631': 'R',
 'sa828691': 'R',
 'sa872787': 'L',
 '18067': 'L',
 'sa876202': 'R',
 'sa873674': 'L',
 '10533': 'R',
 '19955': 'L',
 'sa918920': 'L',
 '18316': 'L',
 '10619': 'R',
 '16448': 'R',
 '9205': 'L',
 'sa875006': 'R',
 'sa738510': 'L',
 '13675': 'R',
 '15766': 'R',
 '19977': 'R',
 '19958': 'R',
 '11338': 'R',
 '17932': 'L',
 '3648': 'R',
 '4418': 'S',
 '16451': 'L',
 '13110': 'R',
 '15988': 'R',
 'sa877497': 'S',
 'sa3007284': 'R',
 '18063': 'R',
 'sa917920': 'L',
 '17689': 'L',
 'sa874730': 'R',
 '16429': 'L',
 '12944': 'L',
 '19514': 'R',
 '10839': 'R',
 '17022': 'R',
 'sa658670': 'L',
 'sa917940': 'S',
 '15490': 'L',
 '16939': 'R',
 '15905': 'R',
 'sa857694': 'L',
 '5486': 'S',
 '14567': 'L',
 'sa832015': 'R',
 '10655': 'L',
 '13897': 'S',
 '16925': 'R',
 '15994': 'L',
 '6013': 'S',
 '19552': 'R',
 '17738': 'L',
 '11761': 'L',
 'sa736940': 'R',
 'sa858039': 'L'}
id_arms_dict = {'19755': 'R',
 'sa737853': 'R',
 'sa828706': 'R',
 '14677': 'R',
 '14120': 'R',
 '15264': 'R',
 '18890': 'R',
 '8302': 'R',
 '15166': 'L',
 'sa876350': 'R',
 'sa873980': 'L',
 'sa915645': 'L',
 'sa3004810': 'L',
 '15507': 'R',
 '17664': 'R',
 '18332': 'R',
 '15256': 'R',
 'sa829037': 'R',
 'sa918414': 'L',
 '17282': 'R',
 '16137': 'L',
 'sa874080': 'L',
 '19261': 'R',
 '14353': 'L',
 'sa3006764': 'R',
 'sa602893': 'R',
 '20151': 'R',
 '14605': 'R',
 'sa738902': 'L',
 '5097': 'L',
 '13218': 'R',
 'sa3007593': 'L',
 'sa917924': 'R',
 'sa3008444': 'R',
 'sa829855': 'R',
 'sa656041': 'R',
 '19457': 'R',
 'sa918036': 'R',
 '9239': 'R',
 'sa659598': 'R',
 'sa874682': 'R',
 'sa874212': 'R',
 'sa3008613': 'R',
 '13287': 'R',
 '16155': 'L',
 'sa828674': 'L',
 '17170': 'L',
 'sa828695': 'R',
 '19459': 'R',
 '16511': 'L',
 'sa916944': 'R',
 '19853': 'R',
 '15100': 'R',
 'sa918091': 'R',
 '19959': 'L',
 '19343': 'L',
 '16164': 'R',
 'sa873274': 'R',
 'sa828669': 'R',
 'sa658943': 'R',
 '15306': 'L',
 'sa738964': 'R',
 '15066': 'R',
 '15514': 'R',
 '11836': 'R',
 '19268': 'R',
 'sa875837': 'R',
 'sa874099': 'L',
 '16507': 'R',
 '19256': 'R',
 'sa922200': 'R',
 '18251': 'R',
 'sa3006700': 'R',
 'sa918372': 'R',
 'sa3006775': 'R',
 '14706': 'L',
 '10343': 'L',
 '16128': 'R',
 '17035': 'L',
 '18325': 'L',
 '11487': 'L',
 'sa875783': 'R',
 '14391': 'L',
 'sa919113': 'R',
 '20387': 'L',
 'sa3005097': 'R',
 '16350': 'L',
 '27460': 'R',
 '4185': 'R',
 '6612': 'L',
 'sa829363': 'R',
 '20218': 'R',
 '9646': 'L',
 '10534': 'R',
 'sa738514': 'R',
 '14332': 'R',
 'sa918271': 'R',
 '3321': 'R',
 'sa829466': 'R',
 '11203': 'L',
 '9895': 'L',
 '19981': 'R',
 '12317': 'R',
 'sa828803': 'R',
 '14672': 'R',
 '9346': 'R',
 '16440': 'R',
 '13654': 'R',
 'sa919117': 'R',
 '15463': 'R',
 'sa874781': 'R',
 '15232': 'L',
 '16208': 'L',
 '4696': 'R',
 '19995': 'R',
 'sa738145': 'R',
 'sa658634': 'R',
 '3374': 'R',
 'sa828689': 'R',
 'sa3006525': 'R',
 '20030': 'L',
 'sa873701': 'L',
 'sa737454': 'R',
 'sa874797': 'R',
 'sa873550': 'R',
 'sa738458': 'R',
 'sa3008070': 'L',
 '8855': 'R',
 'sa829245': 'R',
 'sa917930': 'R',
 'sa915815': 'R',
 'sa874816': 'R',
 '16609': 'R',
 'sa737558': 'L',
 '19614': 'L',
 'sa549399': 'L',
 '16687': 'R',
 'sa873692': 'R',
 '20367': 'R',
 '17878': 'R',
 '18323': 'R',
 'sa880983': 'R',
 '7882': 'R',
 'sa828707': 'R',
 '13942': 'L',
 '15010': 'R',
 'sa658084': 'L',
 'sa829227': 'R',
 '14832': 'R',
 '15538': 'R',
 'sa738901': 'R',
 '11720': 'R',
 'sa918477': 'L',
 '16668': 'R',
 '11243': 'R',
 'sa708446': 'R',
 '12360': 'R',
 '16309': 'R',
 '15492': 'L',
 'sa829588': 'L',
 '10314': 'R',
 '6216': 'R',
 '17755': 'R',
 '13677': 'R',
 'sa830592': 'R',
 '15267': 'R',
 'sa3008225': 'R',
 '18403': 'R',
 'sa3004862': 'L',
 '1157': 'R',
 '8410': 'R',
 '15915': 'R',
 '9492': 'R',
 '8241': 'R',
 'sa827423': 'L',
 '14975': 'R',
 'sa875176': 'R',
 '16074': 'R',
 '17355': 'R',
 'sa875318': 'R',
 '5530': 'L',
 'sa856266': 'R',
 '27458': 'L',
 '16561': 'L',
 '15015': 'R',
 'sa874753': 'R',
 '18496': 'R',
 '2660': 'L',
 'sa3004708': 'R',
 'sa3004011': 'R',
 '9784': 'R',
 'sa3005068': 'L',
 '20039': 'L',
 '13435': 'R',
 '17169': 'L',
 '17292': 'R',
 '19569': 'R',
 'sa857694': 'R',
 'sa885967': 'R',
 '11896': 'R',
 'sa873314': 'R',
 '13533': 'R'}

In [ ]:
class Batter:
    """Class that holds information for a Batter.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L, singles_R, 
                 doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success, bsr, fld):
        """Initializes values for this class
        
        Args:
            team: team name of batter
            name: batter name
            hand: Batting hand of hitter(L, R, or S)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            so_L: fraction of plate appearances ending in a strikeout vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            so_R: fraction of plate appearances ending in a strikeout vs. righties
            sb_rate: approximate percentage of time player will steal base given opportunity
            sb_success: fraction of stolen base attempts ending in success
            bsr: fangraphs stat representing projected runs created by a base runner. Higher number is better
            fld: bsr: fangraphs stat representing projected runs created by a fielder. higher number is better
        """
        self.team = team
        self.name = name
        self.hand = hand
        self.singles_L = singles_L
        self.doubles_L = doubles_L
        self.triples_L = triples_L
        self.home_runs_L = home_runs_L
        self.walks_L = walks_L
        self.so_L = so_L
        self.slg_L = walks_L+singles_L+2*doubles_L+3*triples_L+4*home_runs_L
        self.singles_R = singles_R
        self.doubles_R = doubles_R
        self.triples_R = triples_R
        self.home_runs_R = home_runs_R
        self.walks_R = walks_R
        self.so_R = so_R
        self.slg_R = walks_R+singles_R+2*doubles_R+3*triples_R+4*home_runs_R
        self.sb_rate = sb_rate
        self.sb_success = sb_success
        self.bsr = bsr
        self.fld = fld
        
        #self.cum values represent cumulative (season) totals for a batter
        self.cum_singles_L = 0
        self.cum_doubles_L = 0
        self.cum_triples_L = 0
        self.cum_home_runs_L = 0
        self.cum_walks_L = 0
        self.cum_so_L = 0
        self.cum_pa_L = 0
        self.cum_singles_R = 0
        self.cum_doubles_R = 0
        self.cum_triples_R = 0
        self.cum_home_runs_R = 0
        self.cum_walks_R = 0
        self.cum_so_R = 0
        self.cum_pa_R = 0
        self.cum_runs = 0
        self.cum_rbi = 0
        self.cum_sb = 0
        self.cum_cs = 0
        self.cum_gdp = 0
        self.cum_G = 0
        self.cum_GS = 0
     
    def hits(self):
        """cumulative hits of a batter
        
        Returns:
            int: number of cumulative hits
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L+
               self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg(self):
        """Calculates avg of a player
        
        Returns:
            float: batting average in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(float((self.hits())/((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R))), 3))
        else:
            return 0    
        
    def obp(self):
        """Calculates on base percentage of a player
        
        Returns:
            float: opb in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)>0):
            return(round((self.hits()+(self.cum_walks_L+self.cum_walks_R))/(self.cum_pa_L+self.cum_pa_R), 3))
        else:
            return 0
        
    def slg(self):
        """Calculates slugging percentage of a player
        
        Returns:
            float: slg in decimal form
        """
        if((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)>0):
            return(round(((self.cum_singles_L+self.cum_singles_R)+2*(self.cum_doubles_L+self.cum_doubles_R)+
                           3*(self.cum_triples_L+self.cum_triples_R)+4*(self.cum_home_runs_L+self.cum_home_runs_R))/
                           ((self.cum_pa_L+self.cum_pa_R)-(self.cum_walks_L+self.cum_walks_R)), 3))
        else:
            return 0
        
    def ops(self):
        """Calculates on base plus slugging of a player
        
        Returns:
            float: ops in decimal form
            """
        return(self.slg() + self.obp())
    
    def strikeouts(self):
        """Number of strikeouts of a batter
        
        Returns:
            float: total strikeouts
        """
        return(self.cum_so_L+self.cum_so_R)
    
    def print_stats(self):
        """Prints some stats of a player
        """
        print("Avg: " + str(round(self.avg(), 3)))
        print("OBP: " + str(round(self.obp(), 3)))
        print("SLG: " + str(round(self.slg(), 3)))
        print("OPS: " + str(round(self.ops(), 3)))
        print("HR: " + str((self.cum_home_runs_L+self.cum_home_runs_R)))
              
    def hits_L(self):
        """cumulative hits of a batter against lefties
        
        Returns:
            int: number of hits of batter agianst lefties
        """
        return(self.cum_singles_L+self.cum_doubles_L+self.cum_triples_L+self.cum_home_runs_L)
    
    def avg_L(self):
        """Calculates avg of a player against lefties
        
        Returns:
            float: batting average in decimal form of batter against lefties
        """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(float((self.hits_L())/(self.cum_pa_L-self.cum_walks_L)), 3))
        else:
            return 0    
              
    def obp_L(self):
        """Calculates on base percentage of a player against lefties
        
        Returns:
            float: opb in decimal form of batter against lefties
        """
        if(self.cum_pa_L>0):
            return(round((self.hits_L()+self.cum_walks_L)/self.cum_pa_L, 3))
        else:
            return 0
              
    def slg_L(self):
        """Calculates slugging percentage of a player against lefties
        
        Returns:
            float: slg in decimal form of batter against lefties
            """
        if((self.cum_pa_L-self.cum_walks_L)>0):
            return(round(((self.cum_singles_L+2*self.cum_doubles_L+
                           3*self.cum_triples_L+4*self.cum_home_runs_L))/
                           (self.cum_pa_L-self.cum_walks_L), 3))
        else:
            return 0
              
    def ops_L(self):
        """Calculates on base plus slugging of a player against lefties
        
        Returns:
            float: ops in decimal form of batter against lefties
        """
        return(self.slg_L() + self.obp_L())
              
    def hits_R(self):
        """Cumulative hits of a batter against righties
        
        Returns:
            int: number of hits of batter against righties
        """
        return(self.cum_singles_R+self.cum_doubles_R+self.cum_triples_R+self.cum_home_runs_R)
    
    def avg_R(self):
        """Calculates avg of a player against righties
        
        Returns:
            float: batting average in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(float((self.hits_R())/(self.cum_pa_R-self.cum_walks_R)), 3))
        else:
            return 0    
              
    def obp_R(self):
        """Calculates on base percentage of a player against righties
        
        Returns:
            float: opb in decimal form of batter against righties
        """
        if(self.cum_pa_R>0):
            return(round((self.hits_R()+self.cum_walks_R)/self.cum_pa_R, 3))
        else:
            return 0
              
    def slg_R(self):
        """Calculates slugging percentage of a player against righties
        
        Returns:
            float: slg in decimal form of batter against righties
        """
        if((self.cum_pa_R-self.cum_walks_R)>0):
            return(round(((self.cum_singles_R2*self.cum_doubles_R+
                           3*self.cum_triples_R+4*self.cum_home_runs_R))/
                           (self.cum_pa_R-self.cum_walks_R), 3))
        else:
            return 0
              
    def ops_R(self):
        """Calculates on base plus slugging of a player against righties
        
        Returns:
            float: ops in decimal form of batter against righties
        """
        
        return(self.slg_R() + self.obp_R())
              
class Pitcher(Batter):
    """Class that holds information for a pitcher. Many attributes are similar to those from the Batter class.
    
    Attributes:

    """
    def __init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L, singles_R,
                 doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success, bsr, fld):
        """Initializes values for this class
        
        Args:
            team: team name of pitcher
            name: pitcher name
            hand: Pitching arm of pitcher(L, R)
            singles_L: fraction of plate appearances ending in a single vs. lefties
            doubles_L: fraction of plate appearances ending in a double vs. lefties
            triples_L: fraction of plate appearances ending in a triple vs. lefties
            home_runs_L: fraction of plate appearances ending in a home run vs. lefties
            walks_L: fraction of plate appearances ending in a walk vs. lefties
            singles_RL: fraction of plate appearances ending in a single vs. righties
            doubles_R: fraction of plate appearances ending in a double vs. righties
            triples_R: fraction of plate appearances ending in a triple vs. righties
            home_runs_R: fraction of plate appearances ending in a home run vs. righties
            walks_R: fraction of plate appearances ending in a walk vs. righties
            sb_rate: approximate percentage of time opposing player will steal base given opportunity (set to 0)
            sb_success: fraction of opponent stolen base attempts ending in success (set to 0)
            bsr: not relevant (set to 0)
            fld: not relevant (set to 0)
        """
        
        Batter.__init__(self, team, name, hand, singles_L, doubles_L, triples_L, home_runs_L, walks_L, so_L,
                        singles_R, doubles_R, triples_R, home_runs_R, walks_R, so_R, sb_rate, sb_success, 0, 0)
        self.cum_outs_L = 0
        self.cum_outs_R = 0
        self.current_bf = 0
        self.current_outs = 0
        self.current_runs = 0
        self.finished_half = False
        self.days_off = 100
        self.consecutive_days = 0
    
    def cum_outs(self, hand):
        """Cumulative outs of a pitcher against batters of certain handedness
        
        Args:
            hand: handedness of batter pitcher is facing
        Returns:
            float: value for cumulative outs
        """
        if(hand=='L'):
            return self.cum_outs_L
        elif(hand=='R'):
            return self.cum_outs_R
    
    def ip(self):
        """Calculates innings pitched of a pitcher
        
        Returns:
            float: Innings pitched in normal form(ie. 5.1 for 5 and a third innings pitched)
        """
        ip = (self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3.0
        ip = math.modf(ip)
        ip = round((ip[0]/3.33333 + ip[1]), 2)
        return(ip)  
    
    def era(self):
        """Calculates ERA of a pitcher
        
        Returns:
            float: ERA in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((27*self.cum_runs/(self.cum_outs_L+self.cum_outs_R+self.cum_cs)), 2))
    
    def whip(self):
        """Calculates WHIP of a pitcher
        
        Returns:
            float: WHIP in decimal form
        """
        if((self.cum_outs_L+self.cum_outs_R)==0):
            return 0
        else:
            return(round((((self.cum_walks_L+self.cum_walks_R)+self.hits())/
                          ((self.cum_outs_L+self.cum_outs_R+self.cum_cs)/3)), 2))
        
class Team:
    """Class that holds information for a team.
    
    Attributes:

    """
    def __init__(self, city, league, batter_df, pitcher_df, starters):
        """Initializes values for this class
        
        Args:
            city: city of team (ex. 'Atlanta')
            league: league of team ('AL' or 'NL')
            batter_df: dataframe of batters on a team and their 2019 stats
            pitcher_df: dataframe of pitchers on a team and their 2019 stats
            starters: dict with names of batters in projected starting lineup, rotation, and the closer
            relievers_df: dataframe with relievers of a team and their projected innings pitched
        """
        self.city = city
        self.league = league
        self.batter_df = batter_df
        self.batter_dict= {}
        self.pitcher_df = pitcher_df
        self.pitcher_dict = {}
        self.starters = starters
        self.pitching_staff = pitcher_df
        self.rotation = [None]*5
        self.closer = None
        self.current_pitcher_list = []
        self.available_pitchers_list = []
        self.batting_pitcher = None
        self.current_pitcher = None
        self.batting_index = 0
        self.rotation_index = 0
        self.runners = [None, None, None, 0]    # first base-home plate. 0 for base empty, 1 for runner on base
        self.runs = 0
        self.wins = 0
        self.losses = 0
        self.cum_runs = 0
        self.cum_runs_allowed = 0
        self.pitchers = []
        self.batters = []
        self.runners_dict  = {}
        self.game_no = 0
        avg_stats = self.batter_df.loc[batter_df['Name']=='Avg_Totals']
        self.avg_batter_stats = avg_stats
        
        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['PA_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['PA_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['PA_R'].values[0]
        batter_df = batter_df[batter_df['Name']!='Avg_Totals']

        for index, batter in batter_df.iterrows():   # adding values to each batter and instantiating Batter class
            """
            singles_L = max(0, round(2*batter['1B_L']/batter['PA_L']-avg_singles_L, 3))
            doubles_L = max(0, round(2*batter['2B_L']/batter['PA_L']-avg_doubles_L, 3))
            triples_L = max(0, round(2*batter['3B_L']/batter['PA_L']-avg_triples_L, 3))
            home_runs_L = max(0, round(2*batter['HR_L']/batter['PA_L']-avg_home_runs_L, 3))
            walks_L = max(0, round(2*batter['BB_L']/batter['PA_L']-avg_walks_L, 3))
            so_L = max(0, round(2*batter['SO_L']/batter['PA_L']-avg_so_L, 3))
            singles_R = max(0, round(2*batter['1B_R']/batter['PA_R']-avg_singles_R, 3))
            doubles_R = max(0, round(2*batter['2B_R']/batter['PA_R']-avg_doubles_R, 3))
            triples_R = max(0, round(2*batter['3B_R']/batter['PA_R']-avg_triples_R, 3))
            home_runs_R = max(0, round(2*batter['HR_R']/batter['PA_R']-avg_home_runs_R, 3))
            walks_R = max(0, round(2*batter['BB_R']/batter['PA_R']-avg_walks_R, 3))
            so_R = max(0, round(2*batter['SO_R']/batter['PA_R']-avg_so_R, 3))
            """
            singles_L = round(2*batter['1B_L']/batter['PA_L']-avg_singles_L, 3)
            doubles_L = round(2*batter['2B_L']/batter['PA_L']-avg_doubles_L, 3)
            triples_L = round(2*batter['3B_L']/batter['PA_L']-avg_triples_L, 3)
            home_runs_L = round(2*batter['HR_L']/batter['PA_L']-avg_home_runs_L, 3)
            walks_L = round(2*batter['BB_L']/batter['PA_L']-avg_walks_L, 3)
            so_L = round(2*batter['SO_L']/batter['PA_L']-avg_so_L, 3)
            singles_R = round(2*batter['1B_R']/batter['PA_R']-avg_singles_R, 3)
            doubles_R = round(2*batter['2B_R']/batter['PA_R']-avg_doubles_R, 3)
            triples_R = round(2*batter['3B_R']/batter['PA_R']-avg_triples_R, 3)
            home_runs_R = round(2*batter['HR_R']/batter['PA_R']-avg_home_runs_R, 3)
            walks_R = round(2*batter['BB_R']/batter['PA_R']-avg_walks_R, 3)
            so_R = round(2*batter['SO_R']/batter['PA_R']-avg_so_R, 3)
            
            sb_rate = round((batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R'])/
                            (batter['1B_L']+batter['1B_R']+batter['BB_L']+batter['BB_R']), 3)
            if(batter['CS_L']+batter['CS_R'])==0:
                sb_success = 0
            else:
                sb_success = ((batter['SB_L']+batter['SB_R'])/
                             (batter['SB_L']+batter['SB_R']+batter['CS_L']+batter['CS_R']))

            current_batter = Batter(self.city, batter['Name'], batter['Bats'], singles_L, doubles_L, triples_L, 
                                    home_runs_L, walks_L, so_L, singles_R, doubles_R, triples_R, 
                                    home_runs_R, walks_R, so_R, sb_rate, sb_success, batter['BsR'], batter['Fld'])
            self.batters.append(current_batter)
            self.batter_dict.update({batter['Name']: current_batter})
            
        avg_stats = self.pitcher_df.loc[pitcher_df['Name']=='Avg_Totals']
        self.avg_pitcher_stats = avg_stats
        
        avg_singles_L = avg_stats['1B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_doubles_L = avg_stats['2B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_triples_L = avg_stats['3B_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_home_runs_L = avg_stats['HR_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_walks_L = avg_stats['BB_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_so_L = avg_stats['SO_L'].values[0]/avg_stats['TBF_L'].values[0]
        avg_singles_R = avg_stats['1B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_doubles_R = avg_stats['2B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_triples_R = avg_stats['3B_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_home_runs_R = avg_stats['HR_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_walks_R = avg_stats['BB_R'].values[0]/avg_stats['TBF_R'].values[0]
        avg_so_R = avg_stats['SO_R'].values[0]/avg_stats['TBF_R'].values[0]
        pitcher_df = pitcher_df[pitcher_df['Name']!='Avg_Totals']
        
        for index, pitcher in pitcher_df.iterrows(): # adding values to each pitcher and instantiating Pitcher class
            singles_L = round(2*pitcher['1B_L']/pitcher['TBF_L']-avg_singles_L, 3)
            doubles_L = round(2*pitcher['2B_L']/pitcher['TBF_L']-avg_doubles_L, 3)
            triples_L = round(2*pitcher['3B_L']/pitcher['TBF_L']-avg_triples_L, 3)
            home_runs_L = round(2*pitcher['HR_L']/pitcher['TBF_L']-avg_home_runs_L, 3)
            walks_L = round(2*pitcher['BB_L']/pitcher['TBF_L']-avg_walks_L, 3)
            so_L = round(2*pitcher['SO_L']/pitcher['TBF_L']-avg_so_L, 3)
            
            singles_R = round(2*pitcher['1B_R']/pitcher['TBF_R']-avg_singles_R, 3)
            doubles_R = round(2*pitcher['2B_R']/pitcher['TBF_R']-avg_doubles_R, 3)
            triples_R = round(2*pitcher['3B_R']/pitcher['TBF_R']-avg_triples_R, 3)
            home_runs_R = round(2*pitcher['HR_R']/pitcher['TBF_R']-avg_home_runs_R, 3)
            walks_R = round(2*pitcher['BB_R']/pitcher['TBF_R']-avg_walks_R, 3)
            so_R = round(2*pitcher['SO_R']/pitcher['TBF_R']-avg_so_R, 3)

            current_pitcher = Pitcher(self.city, pitcher['Name'], pitcher['Throws'], singles_L, doubles_L, triples_L, 
                                  home_runs_L, walks_L, so_L, singles_R, doubles_R, triples_R, 
                                  home_runs_R, walks_R, so_R, 0, 0, 0, 0)
            
            self.pitchers.append(current_pitcher)
            self.pitcher_dict.update({pitcher['Name']: current_pitcher})
            
            if(current_pitcher.name==starters['closer']):    # Assigning the team's projected closer to self.closer
                self.closer = current_pitcher

        def make_opening_lineup(batters, starters, league):
            """Making a lineup of batters based on projected lineup
        
            Args:
                batters: dataframe of batters
                starters: dict with names in projected starting lineup
            Returns:
                lineup: list of Batter instances in order of projected lineup
            """
            lineup = []
            
            for starter in starters['lineup']:
                for batter in batters:
                    if(batter.name==starter):
                        lineup.append(batter)
                        
            if(league=='NL'):
                # If not all batters in projected lineup have data, create a batter to be in the lineup
                while(len(lineup)<8):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                                  .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, 0, 0))
                # Adding batter to hit in pitcher's spot
                lineup.append(Batter(self.city, 'pitcher', 'R', .06, .01, .002, .005, .02, .5, .06, .01, 
                                              .002, .005, .02, .5, 0, 0, -5, -5))
            elif(league=='AL'):
                # If not all batters in projected lineup have data, create a batter to be in the lineup
                while(len(lineup)<9):
                    lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                                  .03, .25, .13, .04, .01, .02, .03, .25, .02, .01, 0, 0))             
                
            return lineup
        
        def make_rotation(pitchers):
            """Making a rotation of starting pitchers based on projected rotation
        
            Args:
                pitchers: dataframe of pitchers
                starters: dict with names in projected starting lineup
            Returns:
                rotation: list of Pitcher instances in order of projected rotation
            """
            rotation = []
            
            for starter in starters['rotation']:
                for pitcher in pitchers:
                    if(pitcher.name==starter):
                        rotation.append(pitcher)
                        
            while(len(rotation)<5):
                # If not all pitchers in projected rotation have data, create a pitcher to be in the rotation
                rotation.append(Pitcher(self.city, 'extra'+ str(len(rotation)), 'R', .16, .06, .02, .03, 
                                                 .05, .2, .16, .06, .02, .03, .05, .2, 0, 0))
                
            return rotation
        
        def make_bullpen(pitchers, relievers_df):
            """Making a bullpen of relievers (except the closer) based on projected bullpen
        
            Args:
                pitchers: dataframe of pitchers
                relievers_df: df with names of projected bullpen members and their projected innings pitched
            Returns:
                bullpen_df: df of Pitcher instances and a value that will be used to probabilistically choose relievers
            """
            l_reliever_dict = {}
            l_innings_total = 0
            r_reliever_dict = {}
            r_innings_total = 0
            
            for index, reliever in relievers_df.iterrows():    # Finding relivers from pitcher_df
                for pitcher in pitchers:
                    if(reliever['Name']==pitcher.name and reliever['Name']!=starters['closer']):
                        if(pitcher.hand=='L'):
                            l_reliever_dict.update({pitcher: float(reliever['IP'])})
                            l_innings_total+=float(reliever['IP'])
                        elif(pitcher.hand=='R'):
                            r_reliever_dict.update({pitcher: float(reliever['IP'])})
                            r_innings_total+=float(reliever['IP'])
                        
            relief_df = pd.DataFrame(columns=['Pitcher', 'value'])
            l_value = 0
            for pitcher, ip in l_reliever_dict.items():    # Adding lefty relievers to relief_df with a value
                l_value+=(ip/l_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': l_value}, ignore_index=True)

            r_value = 1
            for pitcher, ip in r_reliever_dict.items():# Adding righty relievers to relief_df with a value
                r_value+=(ip/r_innings_total)
                relief_df = relief_df.append({'Pitcher': pitcher, 'value': r_value}, ignore_index=True)
                      
            return relief_df
            
            
        #self.lineup = make_opening_lineup(self.batters, self.starters, self.league)
        #self.rotation = make_rotation(self.pitchers, self.starters)
        #self.bullpen_df = make_bullpen(self.pitchers, self.relievers_df)
    
    def make_lineup(self, batters, pitcher, league):
        """Making a lineup of batters for a givevn game based on Fangraphs projected plate appearances

        Args:
            batters: dataframe of batters
            starters: dict with names in projected starting lineup
        Returns:
            lineup: list of Batter instances in order of projected lineup
        """
        success = False
        while(not success):
            batter_df = self.batter_df[self.batter_df['Name']!='Avg_Totals']
            unordered_lineup = []
            unordered_df = pd.DataFrame(columns = ['Batter', 'obp', 'slg', 'ops'])
            lineup = [None]*9
            if(league=='NL'):
                pos_list = ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF']
            else:
                pos_list = ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']

            for pos in pos_list:   # Iterating through each position to get a player to be in the lineup
                num = random.randint(0, self.avg_batter_stats[pos].values[0])
                addition = False
                total = 0
                
                while(not addition and total<self.avg_batter_stats[pos].values[0]):
                    for index, row in batter_df.iterrows():
                        batter = self.batter_dict[row['Name']]
                        if(row[pos]+total>num and batter not in unordered_lineup):
                            unordered_lineup.append(batter)
                            
                            if(pitcher.hand=='L'):
                                unordered_df = unordered_df.append(pd.Series([batter, batter.walks_L, batter.slg_L, batter.walks_L+batter.slg_L], index=unordered_df.columns), ignore_index=True)
                            else:
                                unordered_df = unordered_df.append(pd.Series([batter, batter.walks_R, batter.slg_R, batter.walks_R+batter.slg_R], index=unordered_df.columns), ignore_index=True)

                            addition=True
                            break
                        else:
                            total+=row[pos]
                            continue
                        
                
            if(len(unordered_lineup)==len(pos_list)):    # If lineup doesn't have enough batters, restart
                success = True
        
        for num in range(1,3):
            first_ops = unordered_df.loc[unordered_df['ops'].idxmax()]['Batter']
            lineup[num] = first_ops
            unordered_df = unordered_df[unordered_df['Batter']!=first_ops]
        
        first_obp = unordered_df.loc[unordered_df['obp'].idxmax()]['Batter']
        lineup[0] = first_obp
        unordered_df = unordered_df[unordered_df['Batter']!=first_obp]

        for num in range(3,len(pos_list)):
            first_ops = unordered_df.loc[unordered_df['ops'].idxmax()]['Batter']
            lineup[num] = first_ops
            unordered_df = unordered_df[unordered_df['Batter']!=first_ops]
        

        if(league=='NL'):
            if(self.batting_pitcher==None):
                self.batting_pitcher = Batter(self.city, 'pitcher', 'R', .06, .01, -.05, -.05, -.05, .5, .06, .01, 
                                              -.05, -.05, -.05, .5, 0, 0, -5, -5)
                lineup[8] = self.batting_pitcher
                self.batters.append(self.batting_pitcher)
            else:
                lineup[8] = self.batting_pitcher
            
        elif(league=='AL'):
            # If not all batters in projected lineup have data, create a batter to be in the lineup
            while(len(lineup)<9):
                lineup.append(Batter(self.city, 'extra' +str(len(lineup)), 'R', .13, .04, .01, .02, 
                                              .03, .25, .13, .04, .01, .02, .03, .25, .02, .01))             

        for batter in lineup:
            batter.cum_G+=1
            batter.cum_GS+=1
        
        return lineup
    
    def choose_pitcher(self, outs, inning, runs, opp_runs, lineup, lineup_index):
        """Returns a pitcher to pitch for a team given game attributes
        
            Args:
                outs: current number of outs in inning
                runs: number of runs team has
                opp_runs: number of runs opposing team has
                lineup: opposing team's lineup
                lineup_index: place in lineup opposing team is
            Returns:
                instance of Pitcher class to pitch in game
            """
        remove_pitcher = False
        self.available_pitchers_list = []
        for pitcher in self.pitchers:  # Adding pitchers who have not already pitched to a list
            if pitcher not in self.current_pitcher_list and pitcher not in self.available_pitchers_list:
                self.available_pitchers_list.append(pitcher)
            
        # If Pitcher hasn't faced the minimum of 3 batters or finished a hlaf inning, they remain in game
        if(self.current_pitcher.current_bf<3 and self.current_pitcher.finished_half==False): 
            return self.current_pitcher
        
        # Creating variable total that uses game factors to determine if pitcher should get taken out
        total = ((self.current_pitcher.current_bf-self.current_pitcher.current_outs)*3 + 
                 self.current_pitcher.current_outs+self.current_pitcher.current_runs*3)

        if(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3.0>0):
            total+=(self.current_pitcher.current_runs-self.current_pitcher.current_outs/3)*2

        if(self.current_pitcher.hand==lineup[lineup_index].hand):
            total-=3
            
        if(self.current_pitcher.finished_half):
            total+=2
        
        if(outs==2):
            total+=2
        
        if(total<40):
            num = random.uniform(0, (60-total)*100)
        else:
            num = random.uniform(0, (90-total)*4)
            
        if(num<total):
            remove_pitcher = True
        
        # Leave current potcher in probabilistically
        if(not remove_pitcher and self.current_pitcher in self.rotation or total<=8 and self.current_pitcher not 
           in self.rotation):
            return self.current_pitcher
        else:  # If current pitcher has total value greater than threshold, find new reliever
            
            if(inning==8 and outs>0 and self.closer not in self.current_pitcher_list 
               and runs-opp_runs>0 and runs-opp_runs<3): # Put closer in game if game is close and in late innings
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            elif(inning>8 and self.closer not in self.current_pitcher_list and runs-opp_runs>0 and runs-opp_runs<4):
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                # If game not close or game not in late innings, put non-closer reliever in game
                
                left_count = 0    # Number of lefties out of next three batters
                right_count = 0   # Number of righties out of next three batters

                for num in range(3):
                    if(lineup[(lineup_index+num)%9].hand=='L'):
                        left_count+=(1+outs-num)
                    elif(lineup[(lineup_index+num)%9].hand=='R'):
                        right_count+=(1+outs-num)

                if(left_count>right_count):  # If more of the next 3 batters are lefties, hand_indicator = 0
                    hand_indicator = 'L'
                else:                        # If more or equal of the next 3 batters are righties, hand_indicator = 1
                    hand_indicator = 'R'
                
                
                pitcher_df = self.pitcher_df[self.pitcher_df['Name']!='Avg_Totals']
        
                
                reliever_chosen = False
                wrong_hand = 0
                attempts = 0
                while(attempts<200):    # Trying to find a reliever
                    num = random.randint(0, self.avg_pitcher_stats['RP'].values[0])
                    total = 0
                    if(attempts<50):
                        days_off_threshold = 4
                    elif(attempts<100):
                        if(self.closer in self.available_pitchers_list):
                            self.current_pitcher = self.closer
                            self.current_pitcher.cum_G+=1
                            self.current_pitcher_list.append(self.current_pitcher)
                            return self.current_pitcher
                        
                        days_off_threshold = 7
                    else:
                        days_off_threshold = 10
                    
                    for index, row in pitcher_df.iterrows():
                        pitcher = self.pitcher_dict[row['Name']]
                        if(pitcher in self.available_pitchers_list and pitcher!=self.closer):
                            if(row['RP']+total>num and pitcher.consecutive_days<days_off_threshold):
                                if(pitcher.hand==hand_indicator or wrong_hand>1):
                                    self.current_pitcher = pitcher
                                    self.current_pitcher.cum_G+=1
                                    self.current_pitcher_list.append(self.current_pitcher)
                                    return self.current_pitcher
                                if(wrong_hand<2):    # If pitcher chosen doesn't have same hand as most upcoming batters, will try again
                                    wrong_hand+=1
                                    attempts+=1
                                    break
                            else:
                                total+=row['RP']
                                continue
                        else:
                            attempts+=1

            if(self.closer in self.available_pitchers_list):
                self.current_pitcher = self.closer
                self.current_pitcher.cum_G+=1
                self.current_pitcher_list.append(self.current_pitcher)
                return self.current_pitcher
            else:
                pitcher_chosen = False
                attempts = 0
                while(not pitcher_chosen and len(self.available_pitchers_list)>0 and attempts<100):
                    num = random.randint(0, len(self.available_pitchers_list)-1)
                    pitcher = self.available_pitchers_list[num]
                    if(pitcher in self.rotation and attempts<30 and pitcher.days_off<3):
                        attempts+=1
                    elif(pitcher in self.rotation and attempts<60 and pitcher.days_off<2):
                        attempts+=1
                    elif(pitcher in self.rotation and attempts<100 and pitcher.days_off<1):
                        attempts+=1
                    else:
                        self.current_pitcher = pitcher
                        self.current_pitcher.cum_G+=1
                        self.current_pitcher_list.append(self.current_pitcher)
                        return self.current_pitcher

        return self.current_pitcher
    
    def choose_starter(self):
        """Chooses a starting pitcher for a team
        """
        pitcher_df = self.pitcher_df[self.pitcher_df['Name']!='Avg_Totals']
        pitcher_df.sort_values(by=['SP'], ascending=False, inplace=True)
        
        if(self.game_no<6):    # If early in season, pick starter based on projected innings pitched
            for index, row in pitcher_df.iterrows():
                if(index>=self.game_no-1):
                    pitcher = self.pitcher_dict[row['Name']]
                    if(pitcher.days_off>=4):
                        starter_chosen=True
                        pitcher.cum_G+=1
                        pitcher.cum_GS+=1
                        self.rotation[self.rotation_index] = pitcher
                        return pitcher

        starter_chosen = False

        while(not starter_chosen):
            supposed_starter = self.rotation[self.rotation_index]
            starts_projected = int(pitcher_df[pitcher_df['Name']==supposed_starter.name]['SP']/6)
            n = random.randint(0, 40)
            if(n<starts_projected):
                    supposed_starter.cum_G+=1
                    supposed_starter.cum_GS+=1
                    self.rotation[self.rotation_index] = supposed_starter
                    return supposed_starter
            else:
                total = 0
                num = random.randint(0, self.avg_pitcher_stats['SP'].values[0])
                for index, row in pitcher_df.iterrows():
                    pitcher = self.pitcher_dict[row['Name']]
                    if(row['SP']+total>num and pitcher.days_off>=4):
                        pitcher.cum_G+=1
                        pitcher.cum_GS+=1
                        self.rotation[self.rotation_index] = pitcher
                        return pitcher
                    else:
                        total+=row['SP']
                        continue

    def restart(self):
        """Resets values for position in lineup, runners on base, and runs if a game is to be started
        """
        for pitcher in self.current_pitcher_list:
            pitcher.consecutive_days+=1
            pitcher.days_off = 0
            pitcher.current_bf = 0
            pitcher.current_outs = 0
            pitcher.current_runs = 0
            pitcher.finished_half = False
        
        for pitcher in self.available_pitchers_list:
            pitcher.days_off+=1
            pitcher.consecutive_days = 0

        self.runners_dict = {}    
        self.current_pitcher_list = []   
        self.batting_index = 0
        self.current_pitcher = None
        self.starter_pitching = True
        self.runners = [None, None, None, 0]
        self.runs = 0
        
    def reset_runners(self):
        """Resets baserunners after each half inning
        """
        self.runners = [None, None, None, 0]
    
    def cum_stats_df(self):
        """Class that returns dataframes of stats for batters and pitchers
    
        Returns:
            batter_df: dataframe of batter stats
            pitcher_df: dataframe of pitcher stats
        """
        batter_df = pd.DataFrame(columns=['Name', 'G', 'PA', 'AB', 'H', '1B', '2B', '3B', 'HR', 'BB', 'K', 'AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'RBI', 'SB', 'CS'])
        
        for b in self.batters:
            batter_values = [b.name, b.cum_G, b.cum_pa_L+b.cum_pa_R, b.cum_pa_L+b.cum_pa_R-(b.cum_walks_L+b.cum_walks_R), 
                             b.hits(), b.cum_singles_L+b.cum_singles_R, b.cum_doubles_L+b.cum_doubles_R, 
                             b.cum_triples_L+b.cum_triples_R, b.cum_home_runs_L+b.cum_home_runs_R, 
                             b.cum_walks_L+b.cum_walks_R, b.strikeouts(), b.avg(), b.obp(), b.slg(), b.ops(), b.cum_runs, 
                             b.cum_rbi, b.cum_sb, b.cum_cs]
            batter_df = batter_df.append(pd.Series(batter_values, index=batter_df.columns), ignore_index=True)
        batter_df.sort_values(by=['PA'], ascending=False, inplace=True)
        
        pitcher_df = pd.DataFrame(columns=['Name', 'G', 'GS', 'IP', 'K', 'H', 'BB', 'ERA', 'WHIP', 'HR','AVG', 'OBP', 
                                          'SLG', 'OPS', 'R', 'SB', 'CS'])
        
        for p in self.pitchers:
            pitcher_values = [p.name, p.cum_G, p.cum_GS, p.ip(), p.strikeouts(), p.hits(), p.cum_walks_L+p.cum_walks_R, p.era(), p.whip(), 
                              p.cum_home_runs_L+p.cum_home_runs_R, p.avg(), p.obp(), p.slg(), p.ops(),
                              p.cum_runs, p.cum_sb, p.cum_cs]
            pitcher_df = pitcher_df.append(pd.Series(pitcher_values, index=pitcher_df.columns), ignore_index=True)
        pitcher_df.sort_values(by=['IP'], ascending=False, inplace=True) 
        
        return batter_df, pitcher_df
    
class Game:
    """Class that holds information for a Game.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team
                                
    def get_stolen_base_result(self, runners, pitcher, num):
        """stolen base result of a batter
        
        Args:
            runners: list of runners on base
            pitcher: pitcher currently pitching
            num: random number used to determine whether or not batter should steal base
        Returns:
            result of stolen base attempt (or None if no attempt)
        """
        if(runners[0]!=None and runners[1]==None):   # If first base occupied and second base not, steal possible
            if(num>runners[0].sb_rate):    # No stolen base attempt if random number greater than runner sb_rate
                return None
            else:    # Stolen base attempt
                x = random.uniform(0,1)
                if(x<runners[0].sb_success):  # Successful stolen base attempt
                    runners[0].cum_sb+=1
                    pitcher.cum_sb+=1
                    runners[1] = runners[0]
                    runners[0] = None
                    return None
                        
                else:    # Batter caught stealing
                    runners[0].cum_cs+=1
                    pitcher.cum_cs+=1
                    runners[0] = None
                    return "out"
        else:
            return None
        
    def get_result(self, batter, pitcher, outs, runners, num):
        """Using player attributes and a random number, a result is created for a plate appearance
        
        Args:
            batter: instance of batter class
            pitcher: instance of pitcher class
            outs: number of outs in inning
            runners: List of where runners are
            num: random number between 0 and 2
        Returns:
            str: refers to result based on random number (ie "walk")
        """      
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        pitcher.current_bf+=1

        if(batter_against=='L' and pitcher_against=='L'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_L+pitcher.singles_L):
                batter.cum_singles_L+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L):
                batter.cum_triples_L+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L):
                batter.cum_walks_L+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_L+batter.doubles_L+pitcher.doubles_L+batter.triples_L+pitcher.triples_L+batter.home_runs_L+pitcher.home_runs_L
                 +batter.walks_L+pitcher.walks_L+batter.so_L+pitcher.so_L):
                batter.cum_so_L+=1                
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='L'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_L+=1
            if(num<batter.singles_R+pitcher.singles_L):
                batter.cum_singles_R+=1
                pitcher.cum_singles_L+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_L+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L):
                batter.cum_triples_R+=1
                pitcher.cum_triples_L+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_L+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L):
                batter.cum_walks_R+=1
                pitcher.cum_walks_L+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_L+batter.doubles_R+pitcher.doubles_L+batter.triples_R+pitcher.triples_L+batter.home_runs_R+pitcher.home_runs_L
                 +batter.walks_R+pitcher.walks_L+batter.so_R+pitcher.so_L):
                batter.cum_so_R+=1
                pitcher.cum_outs_L+=1
                pitcher.cum_so_L+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_L+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_L+=1
                    return "out"
        elif(batter_against=='L' and pitcher_against=='R'):
            batter.cum_pa_L+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_L+pitcher.singles_R):
                batter.cum_singles_L+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R):
                batter.cum_doubles_L+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R):
                batter.cum_triples_L+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R):
                batter.cum_home_runs_L+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R):
                batter.cum_walks_L+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_L+pitcher.singles_R+batter.doubles_L+pitcher.doubles_R+batter.triples_L+pitcher.triples_R+batter.home_runs_L+pitcher.home_runs_R
                 +batter.walks_L+pitcher.walks_R+batter.so_L+pitcher.so_R):
                batter.cum_so_L+=1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                else:
                    pitcher.cum_outs_R+=1
                    return "out"
        elif(batter_against=='R' and pitcher_against=='R'):
            batter.cum_pa_R+=1
            pitcher.cum_pa_R+=1
            if(num<batter.singles_R+pitcher.singles_R):
                batter.cum_singles_R+=1
                pitcher.cum_singles_R+=1
                return "single"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R):
                batter.cum_doubles_R+=1
                pitcher.cum_doubles_R+=1
                return "double"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R):
                batter.cum_triples_R+=1
                pitcher.cum_triples_R+=1
                return "triple"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R):
                batter.cum_home_runs_R+=1
                pitcher.cum_home_runs_R+=1
                return "home_run"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R):
                batter.cum_walks_R+=1
                pitcher.cum_walks_R+=1
                return "walk"
            elif(num<batter.singles_R+pitcher.singles_R+batter.doubles_R+pitcher.doubles_R+batter.triples_R+pitcher.triples_R+batter.home_runs_R+pitcher.home_runs_R
                 +batter.walks_R+pitcher.walks_R+batter.so_R+pitcher.so_R):
                batter.cum_so_R+=1
                pitcher.cum_outs_R+=1
                pitcher.cum_so_R+=1
                return "k"
            else:
                 if(outs<2 and runners[0]!=None and num>(1.9+batter.bsr/10)):
                    batter.cum_gdp+=1
                    pitcher.cum_outs_R+=2
                    pitcher.cum_gdp+=1
                    return "gdp"
                 else:
                    pitcher.cum_outs_R+=1
                    return "out"
              
    def move_runners(self, team, batter, pitcher, outs, result):
        """Based on the result of a batter's plate appearance, runners are moved and runs may be added to team's total 
        
        Args:
            team: instance team that the batter was on
            batter who is hitting
            pitcher: pitcher pitching to team
            outs: number of outs
            result: str of result for batter (ie "walk")
        """
        
        made_out = False
        
        if(pitcher.hand=='L'):
            batter_against = 'L'
        elif(pitcher.hand=='R'):
            batter_against = 'R'
              
        if(batter.hand=='L'):
            pitcher_against = 'L'
        elif(batter.hand=='R'):
            pitcher_against = 'R'
        elif(batter.hand=='S'):
            if(pitcher.hand=='L'):
                pitcher_against='R'
            elif(pitcher.hand=='R'):
                pitcher_against='L'
        
        if(result=='single'):
            second_to_home = False    # True if runner from second attempts to score
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                team.lineup[team.batting_index].cum_rbi+=1
                team.runners[3]+=1
                
            if(team.runners[1]!=None):  
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                if(num<((batter.bsr*2)+45+(outs*10))):
                    second_to_home = True
                    new_num = random.uniform(0,100)
                    if(new_num<3):
                        if(pitcher_against=='L'):
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[1].cum_runs+=1
                        team.runners_dict[team.runners[1]].cum_runs+=1
                        team.runners_dict[team.runners[1]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1
                else:
                    team.runners[2] = team.runners[1]
                    
            if(team.runners[0]!=None and second_to_home):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                if(num<((batter.bsr*3)+30)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[2] = team.runners[0] 
                else:
                    team.runners[1] = team.runners[0]
                
            team.runners[0] = batter
            team.runners_dict.update({batter: pitcher})

        elif(result=='double'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            if(team.runners[0]!=None):
                num = random.uniform(0, 100)    # Random number that will be used to move runners probabilistically
                if(num<((batter.bsr*4)+45)):
                    new_num = random.uniform(0,100)
                    if(new_num<2):
                        if(pitcher_against=='L'):
                            pitcher.cum_outs_L+=1
                            made_out = True
                        else:
                            pitcher.cum_outs_R+=1
                            made_out = True
                    else:
                        team.runners[0].cum_runs+=1
                        team.runners_dict[team.runners[0]].cum_runs+=1
                        team.runners_dict[team.runners[0]].current_runs+=1
                        batter.cum_rbi+=1
                        team.runners[3]+=1 
                else:
                    team.runners[2] = team.runners[0]
                
            team.runners[1] = batter
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='triple'):
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = batter
            team.runners[1] = None
            team.runners[0] = None
            team.runners_dict.update({batter: pitcher})

        elif(result=='home_run'):
            batter.cum_runs+=1
            pitcher.cum_runs+=1
            pitcher.current_runs+=1
            batter.cum_rbi+=1
            team.runners[3]+=1
            
            if(team.runners[2]!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[1]!=None):
                team.runners[1].cum_runs+=1
                team.runners_dict[team.runners[1]].cum_runs+=1
                team.runners_dict[team.runners[1]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            if(team.runners[0]!=None):
                team.runners[0].cum_runs+=1
                team.runners_dict[team.runners[0]].cum_runs+=1
                team.runners_dict[team.runners[0]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
                
            team.runners[2] = None
            team.runners[1] = None
            team.runners[0] = None

        elif(result=='walk'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            team.runners_dict.update({batter: pitcher})
            
            if(temp_0!=None and temp_1!=None and temp_2!=None):
                team.runners[2].cum_runs+=1
                team.runners_dict[team.runners[2]].cum_runs+=1
                team.runners_dict[team.runners[2]].current_runs+=1
                batter.cum_rbi+=1
                team.runners[3]+=1
            
            team.runners[0] = batter
            
            if(temp_0!=None):
                team.runners[1] = temp_0
                
                if(temp_1!=None):
                    team.runners[2] = temp_1
                    
                        
        elif(result=="gdp"):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            rand_num = random.uniform(0,1)

            if(temp_1==None and temp_2==None):    # If only runner on first
                team.runners[0]==None
            elif(temp_1!=None and temp_2==None):    # If runners on first and second
                if(rand_num<.02):   # Outs made at first and third
                    team.runners[1] = temp_0
                    team.runners[0] = None
                elif(rand_num<.08):   # Outs made at third and second
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                else:
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1==None and temp_2!=None):    # If runners on first and third
                if(rand_num<.01):   # Batter goes to first
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                elif(rand_num<.02):   # Batter goes to second
                    team.runners[0] = None
                    team.runners[1] = batter
                    team.runners_dict.update({batter: pitcher})
                    team.runners[2] = None
                else: # Runner scores, bases empty
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[2] = None
                    team.runners[1] = None
                    team.runners[0] = None
            elif(temp_1!=None and temp_2!=None):    # If bases loaded
                if(rand_num<.6):   # Outs made at second and first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    team.runners[3]+=1
                    team.runners[0] = None
                    team.runners[1] = None
                    team.runners[2] = temp_1
                elif(rand_num<.9):   # Outs made at home and first
                    team.runners[1] = temp_0
                    team.runners[2] = temp_1
                    team.runners[0] = None
                else:    # Outs made at third and home
                    team.runners[2] = None
                    team.runners[1] =temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                    
        elif(result=='out'):
            temp_2 = team.runners[2]
            temp_1 = team.runners[1]
            temp_0 = team.runners[0]
            
            if(temp_0!=None and temp_1==None and temp_2==None):
                rand_num = random.uniform(0,1)
                
                if(rand_num<(.3+batter.bsr/15)):   # If batter hits ground ball and reaches on fielder's choice
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4):    # If batter hits ground ball and is out but runner on first moves to second
                    team.runners[1] = temp_1
                    team.runners[0] = None
            elif(temp_0!=None and temp_1!=None and temp_2==None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.1+temp_1.bsr/15)):   # If batter hits sacrifice fly moving runner to third
                    team.runners[2] = temp_1
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, guy on second goes to third, guy on first is out, batter goes to first
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35):    # If batter hits ball, guy on second goes to third, guy on first goes to second, batter makes out
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = None
            elif(temp_0!=None and temp_1==None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.2+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.05):
                        team.runners[1] = temp_0
                    
                    team.runners[2] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home
                    team.runners[2] = None
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.35+batter.bsr/15):    # If batter hits ball, guy on third scores, guy on first out, batter ends up on first
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0==None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.24+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    
                    team.runners[1] = None
                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runner on second advances
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.33):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
            elif(temp_0!=None and temp_1!=None and temp_2!=None):
                rand_num = random.uniform(0,1)
                if(rand_num<(.24+temp_2.bsr/15)):   # If batter hits sacrifice fly scoring runner
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    
                    if(rand_num<.15):
                        team.runners[2] = temp_1
                    if(rand_num<.06):
                        team.runners[1] = temp_0

                elif(rand_num<.3):    # If batter hits ground ball, out made at home, runners advance
                    team.runners[2] = temp_1
                    team.runners[1] = temp_0
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})
                elif(rand_num<.4+batter.bsr/15):    # If batter hits ground ball, out made at home, runner on second stays
                    team.runners[2].cum_runs+=1
                    team.runners_dict[team.runners[2]].cum_runs+=1
                    team.runners_dict[team.runners[2]].current_runs+=1
                    batter.cum_rbi+=1
                    team.runners[3]+=1
                    team.runners[2] = temp_1
                    team.runners[1] = None
                    team.runners[0] = batter
                    team.runners_dict.update({batter: pitcher})

        if(team.runners[3]>0):
            team.runs+=team.runners[3]
            team.runners[3] = 0
            
        if(made_out):
            return 'out'

    def print_result(self):
        """Printed score of a game
        """ 
        print("final score: " + self.away_team.city + " " + str(self.away_team.runs) + " " 
              + self.home_team.city + " " + str(self.home_team.runs))     
    
    def print_record(self):
        """Printed record of teams
        """ 
        print(str(self.away_team.wins) + "-" + str(self.away_team.losses))
        print(str(self.home_team.wins) + "-" + str(self.home_team.losses))     
   
    def play(self):
        """A game between two instances of the team class is played
        """ 
        self.away_team.restart()
        self.away_team.game_no+=1
        self.away_team.rotation_index = (self.away_team.game_no-1)%5
        self.home_team.restart() 
        self.home_team.game_no+=1
        self.home_team.rotation_index = (self.home_team.game_no-1)%5
        inning = 1
        game_over = False
        
        while(inning<10 or self.away_team.runs==self.home_team.runs):
            for side in [self.away_team, self.home_team]:
                if(inning==9 and side==self.home_team and self.home_team.runs>self.away_team.runs):
                    game_over = True
                    break
                    
                outs = 0
                
                if(inning==1):
                    # Determining starting pitchers
                    if(side==self.away_team):
                        pitcher = self.home_team.choose_starter()
                        self.home_team.current_pitcher_list.append(pitcher)
                        self.home_team.current_pitcher = pitcher
                        self.away_team.lineup = self.away_team.make_lineup(self.away_team.batters, self.home_team.current_pitcher, self.home_team.league)
                    elif(side==self.home_team):
                        pitcher = self.away_team.choose_starter()
                        self.away_team.current_pitcher_list.append(pitcher)
                        self.away_team.current_pitcher = pitcher
                        self.home_team.lineup = self.home_team.make_lineup(self.home_team.batters, self.away_team.current_pitcher, self.home_team.league)

                while(outs<3):
                    # Choosing a pitcher for a team based on game factors
                    if(side==self.away_team):
                        pitcher = self.home_team.choose_pitcher(outs, inning, side.runs, self.home_team.runs, 
                                                                side.lineup, side.batting_index)
                    elif(side==self.home_team):
                        pitcher = self.away_team.choose_pitcher(outs, inning, side.runs, self.away_team.runs, 
                                                                side.lineup, side.batting_index)
                    
                    # Determining whether stolen base should be attempted
                    stolen_base_num = random.uniform(0,1)
                    stolen_base_result = self.get_stolen_base_result(side.runners, pitcher, stolen_base_num)
                    
                    if(stolen_base_result=='out'):   # If failed stolen base, out is recorded
                        pitcher.current_outs+=1
                        outs+=1
                                
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.runners_dict = {}
                            break
                    
                    # Determining outcome of at bat
                    result_num = random.uniform(0,2)
                    result = self.get_result(side.lineup[side.batting_index], pitcher, outs, side.runners, result_num)

                    if(result=='out' or result=='k' or result=='sf'):    # Batter made an out
                        pitcher.current_outs+=1
                        outs+=1
                        
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    elif(result=='gdp'):    # Batter made an out
                        pitcher.current_outs+=2
                        outs+=2
                        
                        if(outs>2):   # If outs = 3, end of inning
                            pitcher.finished_half = True
                            side.batting_index+=1    # Gettting next batter in lineup
                            side.batting_index = side.batting_index%9
                            side.runners_dict = {}
                            break
                        else:
                            self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                            
                    else:    # Batter reached, runners are moved
                        running_result = self.move_runners(side, side.lineup[side.batting_index], pitcher, outs, result)
                        if(running_result=='out'):    # Baserunner made an out
                            pitcher.current_outs+=1
                            outs+=1
                            
                            if(outs>2):   # If outs = 3, end of inning
                                pitcher.finished_half = True
                                side.batting_index+=1    # Gettting next batter in lineup
                                side.batting_index = side.batting_index%9
                                side.runners_dict = {}
                                break
                        
                        if(inning>9 and self.away_team.runs<self.home_team.runs):
                            game_over = True
                    
                    if(game_over):    # If game is over, escape game loop
                        break
                        
                    side.batting_index+=1    # Gettting next batter in lineup
                    side.batting_index = side.batting_index%9
                    
                if(game_over):    # If game is over, escape game loop
                    break
                        
                side.reset_runners()    # Runners are reset after half inning
            

            if(game_over):    # If game is over, escape game loop
                break   
                
            inning+=1
        
        # After game, updating team records and number of cumulative runs
        if(self.away_team.runs>self.home_team.runs):
            self.away_team.wins+=1
            self.home_team.losses+=1
        elif(self.away_team.runs<self.home_team.runs):
            self.away_team.losses+=1
            self.home_team.wins+=1
        
        self.away_team.cum_runs+=self.away_team.runs
        self.away_team.cum_runs_allowed+=self.home_team.runs
        self.home_team.cum_runs+=self.home_team.runs
        self.home_team.cum_runs_allowed+=self.away_team.runs             
        
class Season(Game):
    """Class that holds information for a Season of games.
    
    Attributes:

    """
    
    def __init__(self, away_team, home_team):
        """Initializes values for this class
        
        Args:
            away_team: away team name
            home_team: home team name
        """
        self.away_team = away_team
        self.home_team = home_team

    def simulate_season(self, games):
        """A season of games is simulated
        
        Args:
            games: number of games to be played
        """
        for game in range(1, games+1):
            g = Game(self.away_team, self.home_team)

            g.play() 
            
    def reset_season(self, teams):
        """Reinitializing teams
        
        Args:
            teams, Team instances to reinitialize
        """
        for team in teams:
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters, team.relievers_df)
 
class FullSeason(Game):
    """Class that holds information for a Season of games
    
    Attributes:

    """
    
    def __init__(self, schedule):
        """Initializes values for this class
        
        Args:
            schedule: dataframe of season schedule
        """
        self.schedule = schedule

    def simulate_season(self):
        """A season of games is simulated
        """
        games = 0
        
        for date in self.schedule['date'].unique():
            for acronym, city in acronym_to_city_dict.items():
                away_team = team_dict[acronym_to_city_dict[acronym]]
                home_team_acronym = self.schedule.loc[self.schedule['date']==date][acronym].values[0]

                if(not pd.isna(home_team_acronym)):
                    home_team = team_dict[acronym_to_city_dict[home_team_acronym]]
                    g = Game(away_team, home_team)
                    g.play()
                    games+=1
                    print(games)
  
    def reset_season(self):
        """Reinitializing teams
        """
        for city_name, team in team_dict.items():
            team.__init__(team.city, team.league, team.batter_df, team.pitcher_df, team.starters)   
            
    def print_record(self):
        """Printing records
        """
        for city_name, team in team_dict.items():
            print(team.city + ': ' + str(team.wins) + "-" + str(team.losses))
            
    def league_stats(self):
        """Accumulating stats for the entire league
        """
        pi = pd.DataFrame()
        ba = pd.DataFrame()
        for k, v in nickname_to_city_dict.items():

            a, b = team_dict[v].cum_stats_df()

            if(pi.empty):
                pi = b
            else:
                pi = pi.append(b)

            if(ba.empty):
                ba = a
            else:
                ba = ba.append(a)
        
        pi = pi.sort_values(by=['K'], ascending=False)
        pi.drop_duplicates(inplace=True)
        ba = ba.sort_values(by=['HR'], ascending=False)
        ba.drop_duplicates(inplace=True)
        
        return ba, pi

In [ ]:
team_dict = create_teams(batter_stats_df, pitcher_stats_df)

In [ ]:
s = FullSeason(schedule)
s.reset_season()
s.simulate_season()
s.print_record()

In [ ]:
a, b = s.league_stats()


In [ ]:
a[a['Name']=='pitcher']

In [ ]:
for n in range(162):
    g = Game(team_dict['Los Angeles Angels'], team_dict['Atlanta'])
    g.play()
    g.print_result()

In [ ]:
for num in range(1,1000):
    g = Game(team_dict['New York Yankees'], team_dict['Atlanta'])
    g.play()
    g.print_result()

In [ ]:
#team_dict_df = create_teams(batter_stats_df, pitcher_stats_df, starters_dict, relievers_dict)
for k, v in nickname_to_city_dict.items():
    for k2, v2 in nickname_to_city_dict.items():
        if(v!=v2):
            print(v)
            print(v2)
            g = Game(team_dict[v], team_dict[v2])
            g.play()
            g.print_result()

In [ ]:
pi = pd.DataFrame()
ba = pd.DataFrame()
for k, v in nickname_to_city_dict.items():
    
    a, b = team_dict[v].cum_stats_df()

    if(pi.empty):
        pi = b
    else:
        pi = pi.append(b)
        print(pi)
    if(ba.empty):
        ba = a
    else:
        ba = ba.append(a)
        
#pi.drop_duplicates(inplace=True)
#ba.drop_duplicates(inplace=True)

In [ ]:
pi



In [ ]:
a, b = team_dict['Arizona'].cum_stats_df()
b

In [ ]:
def convert_string(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(str(row[column]))
        else:
            return('0')

def convert_int(row, column):
    
    if(column in row.index):
         if(row[column]!=''):
            return(int(row[column]))
         else:
            return(0)
        
def convert_float(row, column):
    
    if(column in row.index):
        if(row[column]!=''):
            return(float(row[column]))
        else:
            return(0.0)
             
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
  
def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def column_type(batter_stats, pitcher_stats):
    string_columns = ['Season', 'Name', 'Bats', 'Tm', 'playerId']
    int_columns = ['G_R', 'PA_R', 'AB_R',
                   'H_R', '1B_R', '2B_R', '3B_R', 'HR_R', 'R_R', 'RBI_R', 'BB_R', 'IBB_R','SO_R', 'HBP_R', 'SF_R',
                   'SH_R', 'GDP_R', 'SB_R', 'CS_R', 'G_L', 'PA_L', 'AB_L', 'H_L', '1B_L', '2B_L', '3B_L', 'HR_L', 
                   'R_L', 'RBI_L', 'BB_L', 'IBB_L', 'SO_L', 'HBP_L', 'SF_L', 'SH_L', 'GDP_L', 'SB_L', 'CS_L']
    float_columns = ['AVG_R', 'AVG_L']

    for column in batter_stats.columns:
        if(column in string_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_string(row, column), axis=1)
        elif(column in int_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)
        elif(column in float_columns):
            batter_stats[column] = batter_stats.apply(lambda row: convert_float(row, column), axis=1)

    string_columns = ['Season', 'Name', 'Throws', 'Team', 'playerId']
    int_columns = ['IP_L', 'TBF_L', 'H_L', '1B_L', '2B_L', '3B_L', 'R_L', 'ER_L', 'HR_L', 'BB_L', 'IBB_L', 
                   'HBP_L','SO_L', 'IP_R', 'TBF_R', 'H_R', '2B_R', '3B_R', 'R_R', 'ER_R', 'HR_R', 'BB_R', 'IBB_R', 
                   'HBP_R', 'SO_R']
    float_columns = ['ERA_L', 'AVG_L', 'OBP_L', 'SLG_L', 'wOBA_L', 'ERA_R', 'AVG_R', 'OBP_R', 'SLG_R', 'wOBA_R']


    for column in pitcher_stats.columns:
        if(column in string_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_string(row, column), axis=1)
        elif(column in int_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)
        elif(column in float_columns):
            pitcher_stats[column] = pitcher_stats.apply(lambda row: convert_float(row, column), axis=1)     
            
    return batter_stats, pitcher_stats
    
def date_formatter(row):
    """Formats dates
    Args:
        row: row of data
    Returns:
        date in proper form('mm/dd/yyyy')
    """
    if row.date == '':
        return(row.date)
    else:
        split_date = row.date.split('-')
        day = split_date[0]
        day = day.zfill(2)
        month = strptime(split_date[1],'%b').tm_mon
        date = str(month) +  '/' + day + '/' + '2020'
        date = datetime.datetime.strptime(date, '%m/%d/%Y')
        date = date.strftime('%m/%d/%Y')
        
    return date
    
def unknown_player_id(new_player_id, pitchers=False):
    """Finds batting or pitching side of players who do not have data in original stats dataframes
    
    Args:
        new_player_id: dict with player names as keys and their ID's as values
        pitchers: True if dict is of pitchers. Default=False
    Returns:    
        id_arm_dict: dict with ID's as keys and arm or batting side as values
    """

    p_dict = {}
    url_dict = {}
    id_arm_dict = {}
    url_start = 'https://www.fangraphs.com/players/'
    pos_modifier = 0
    
    if(pitchers):
        pos_modifier = 2
    
    
    for k, v in new_player_id.items():
        k = k.replace('\xa0', '')
        k = k.replace(' ', '-')
        p_dict.update({k: v})
        url_dict.update({v: url_start+k+'/'+v})


    for ID, url in url_dict.items():
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        for tag in soup.findAll('div', attrs={'class': 'player-info-box-item'}):

            if(tag.get_text()!='' and tag.get_text().split()[0]=='Bats/Throws:'):
                hand = tag.get_text().split(' ')[1][pos_modifier]
                if(hand=='B'):
                    hand = 'S'
                id_arm_dict.update({ID: hand})

    return id_arm_dict
      

In [ ]:
def batter_stats():

    url = 'http://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2019/start/'
    links = [url+str(i) for i in range(1, 350, 50)]
    headers = {'User-Agent': header_name}

    batter_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            batter_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        batter_stats = pd.concat([batter_stats, df], ignore_index=True)

    return batter_stats

def pitcher_stats():
    url = 'http://www.espn.com/mlb/history/leaders/_/type/pitching/breakdown/season/year/2019/sort/wins/start/'
    links = [url+str(i) for i in range(1, 500, 50)]
    headers = {'User-Agent': header_name}

    pitcher_stats = pd.DataFrame()

    for (num, link) in enumerate(links):
        source = requests.get(link, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        table = soup.find('table', attrs={'class': 'tablehead'})
        if(num==0):
            column_headers = []
            table_headers = table.find('tr', attrs={'class': 'colhead'})
            for header_value in table_headers.findAll('td'):
                column_headers.append(header_value.get_text())

            column_headers[0] = 'ID'
            pitcher_stats = pd.DataFrame(columns=column_headers)

        df = pd.DataFrame(columns=column_headers)            


        for player in table.find_all('tr', attrs={'class': re.compile('row player-10-')}):
            player_values = []
            player_values.append(player['class'][1].split('-')[2])

            values =  player.findAll('td')
            values.pop(0)
            for value in values:
                player_values.append(value.get_text())

            df = df.append(pd.Series(player_values, index=df.columns), ignore_index=True)

        pitcher_stats = pd.concat([pitcher_stats, df], ignore_index=True)

    return pitcher_stats

def projected_starters():

    url = 'https://www.mlb.com/news/projecting-every-mlb-lineup-rotation'
    headers = {'User-Agent': header_name}
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    starters = {}

    for body in soup.findAll('div', attrs={'class': 'article-item__body'}):
        pitchers = []
        batters = []
        closer = None

        for team in body.findAll('p'):
            if(team.strong and team.strong.get_text().lower() in nickname_to_city_dict.keys()):
                current_team = team.strong.get_text().lower()
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    batters.append(unidecode.unidecode(entry.lstrip()))
                
            elif(team.strong and team.strong.get_text()=='Pitchers'):
                a = team.get_text().split(')')
                b = []
                c = []
                d = []

                for entry in a:
                    b.append(str(entry.split('/')[0]))
                for entry in b:
                    c.append(str(entry.split(',')[0]))
                for entry in c:
                    d.append(entry.split(' or ')[0])
            
                for entry in d[1:]:
                    pitchers.append(unidecode.unidecode(entry.lstrip()))
            elif(team.strong and team.strong.get_text()=='Closer:'):
                closer = unidecode.unidecode(team.find(['forge-entity', 'a']).get_text())
            else:
                continue

            if(closer!=None):
                starters.update({nickname_to_city_dict[current_team]: 
                                {'lineup': batters, 'rotation': pitchers, 'closer': closer}})
                pitchers = []
                batters = []
                closer = None
                
    return starters

def projected_relievers():
    league_relief_dict = {}
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='

    for num in range(1,31):
        url_list.append(url_start+str(num))

    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        relief_df = pd.DataFrame(columns=['Name', 'IP'])


        for a_tag in soup.findAll('a', attrs={'href': '#RP'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')
            relief_table = a_tag.find_next('table')

            for row in relief_table.findAll('tr', attrs={'class': 'depth_reg'}):
                name_tag = row.find_next('td')
                name = name_tag.get_text()
                ip = name_tag.find_next('td').get_text()
                relief_df = relief_df.append({'Name': name, 'IP': ip}, ignore_index=True)

        league_relief_dict.update({nickname_to_city_dict[str(team_name.lower())]: relief_df})
        
    return league_relief_dict

def create_player_id_dict(batter_stats, pitcher_stats):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    team_dict = {}
    new_batter_id = {}
    new_pitcher_id = {}

    for num in range(1,31):
        url_list.append(url_start+str(num))
    #url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=16', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=9']
    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')

        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
            pos = a_tag.get_text()

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')


            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]
                values_list = []

                for value in row.findAll('td'):
                    values_list.append(value.get_text())

                    if(len(values_list)==1):
                        values_list.append(player_ID)

                pos_df = pos_df.append(pd.Series(values_list, index=pos_df.columns), ignore_index=True)   

            positional_values_dict.update({pos: pos_df})   

        batter_df = pd.DataFrame()    
        pos_dict = {}

        for pos, df in positional_values_dict.items():
            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']):
                for index, row in df.iterrows():
                    if(row['ID'] in batter_stats['playerId'].values.tolist()):
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_batter_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

                            
            elif(pos in ['SP', 'RP']):
                for index, row in df.iterrows():
                    if(row['ID'] in pitcher_stats['playerId'].values.tolist()):
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            continue
                    else:
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            continue
                        else:
                            new_pitcher_id.update({row['Name']: row['ID']})
                            print(row['Name']+ '  ' + pos)

    return new_batter_id, new_pitcher_id         

def create_teams(batter_stats, pitcher_stats):
    """Creates Team Instances with dataframes of player values of a given team
        
        Args:
            batter_stats: df of batter stats
            pitcher_stats: df of pitcher stats
        Returns:
            team_dict: dict with city of team as key and Team instance as value
        """
    url_list = []
    url_start = 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid='
    team_dict = {}
    new_player_id = {}

    for num in range(1,31):
        url_list.append(url_start+str(num))
    #url_list = ['https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=16', 'https://www.fangraphs.com/depthcharts.aspx?position=ALL&teamid=1']
    for url in url_list:
        headers = {'User-Agent': header_name}
        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')
        positional_values_dict = {}
        pitcher_df = pd.DataFrame()
        batter_df = pd.DataFrame()

        for a_tag in soup.findAll('a', attrs={'href': '#C'}):
            team_name = a_tag.find_next('span').get_text()
            team_name = team_name.replace('\xa0', '')

        for a_tag in soup.findAll('a', attrs={'href': ['#C', '#1B', '#2B', '#SS', '#3B', '#LF', '#CF', '#RF', '#DH', '#SP', '#RP', '#ALL']}):
            pos = a_tag.get_text()

            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH', 'ALL Batters']):
                column_names = ['Name', 'ID', 'PA', 'AVG', 'OBP', 'SLG', 'wOBA', 'Bat', 'BsR', 'Fld', 'WAR']
            else:
                column_names = ['Name', 'ID', 'IP', 'K/9', 'BB/9', 'HR/9', 'BABIP', 'LOB', 'ERA', 'FIP', 'WAR']

            pos_df = pd.DataFrame(columns=column_names)

            table = a_tag.find_next('table')


            for row in table.findAll('tr', attrs={'class': 'depth_reg'}):
                info = row.a['href']
                elements = info.split('=')
                player_ID = elements[1].split('&')[0]
                values_list = []

                for value in row.findAll('td'):
                    values_list.append(value.get_text())

                    if(len(values_list)==1):
                        values_list.append(player_ID)

                pos_df = pos_df.append(pd.Series(values_list, index=pos_df.columns), ignore_index=True)   

            positional_values_dict.update({pos: pos_df})   

        batter_df = pd.DataFrame()    
        pos_dict = {}

        for pos, df in positional_values_dict.items():
            if(pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']):
                for index, row in df.iterrows():
                    if(row['ID'] in batter_stats['playerId'].values.tolist()):
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            batter_df.loc[batter_df['playerId']==row['ID'], pos] = int(row['PA'])
                        else:
                            stats = batter_stats.loc[batter_stats['playerId']==row['ID']]
                            stats[pos] = int(row['PA'])
                            stats['BsR'] = float(row['BsR'])
                            stats['Fld'] = float(row['Fld'])
                            col_headers = stats.columns

                            if(batter_df.empty):
                                batter_df = pd.DataFrame(columns=col_headers)

                            batter_df = batter_df.append(stats, ignore_index=True)
                    else:
                        if(not batter_df.empty and row['ID'] in batter_df['playerId'].values.tolist()):
                            batter_df.loc[batter_df['playerId']==row['ID'], pos] = int(row['PA'])
                        else:
                            player_df = positional_values_dict['ALL Batters']
                            bats = id_bats_dict[row['ID']]

                            new_player_id.update({row['Name']: row['ID']})
                            player_values = player_df.loc[player_df['ID']==row['ID']]

                            if(int(player_values['PA'])<100):
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'], 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Bats': bats, 'G_L': 60, 'PA_L': 220, 'AB_L': 200, 'H_L': 50, '1B_L': 34, '2B_L': 10, 
                                                   '3B_L': 1, 'HR_L': 5, 'R_L': 25, 'RBI_L': 25, 'BB_L': 20, 'IBB_L': 0, 
                                                   'SO_L': 50, 'HBP_L': 3, 'SF_L': 2, 'SH_L': 0, 'GDP_L': 3, 'SB_L': 5, 
                                                   'CS_L': 1, 'AVG_L': .250, 'G_L': 60, 'PA_R': 220, 'AB_R': 200, 'H_R': 50, 
                                                   '1B_R': 34, '2B_R': 10, '3B_R': 1,'HR_R': 5, 'R_R': 25, 'RBI_R': 25, 
                                                   'BB_R': 20, 'IBB_R': 0, 'SO_R': 50, 'HBP_R': 3,'SF_R': 2, 'SH_R': 0, 'GDP_R': 3, 
                                                   'SB_R': 5, 'CS_R': 1, 'AVG_R': .250, 'BsR': 0, 'Fld': 0, pos: int(row['PA'])})

                            else:
                                if(bats=='L'):
                                    multiplier = 1.05
                                elif(bats=='R'):
                                    multiplier = 1/1.05
                                else:
                                    multiplier = 1

                                pa = int(player_values['PA'])
                                obp = float(player_values['OBP'])
                                avg = float(player_values['AVG'])
                                slg = float(player_values['SLG'])
                                
                                on_base = int(round(obp*pa))
                                G_L = int(math.floor(pa*1/(8*multiplier)))
                                G_R = int(math.floor(pa*multiplier*1/4))
                                PA_L = int(math.floor(pa*1/(3*multiplier)))
                                PA_R = pa-PA_L
                                R_L = int(math.floor(PA_L*.15))
                                R_R = int(math.floor(PA_R*.15))
                                RBI_L = int(math.floor(PA_L*.15))
                                RBI_R = int(math.floor(PA_R*.15))
                                HBP_L = int(round(PA_L/100))
                                HBP_R = int(round(PA_R/100))
                                SF_L = int(round(PA_L/180))
                                SF_R = int(round(PA_R/180))
                                SH_L = int(round(PA_L/300))
                                SH_R = int(round(PA_R/300))
                                GDP_L = int(round(PA_L/50))
                                GDP_R = int(round(PA_R/50))
                                SB_L = round(max(0, float(player_values['BsR'])*8)) + round(PA_L/100)
                                SB_R = round(max(0, float(player_values['BsR'])*10)) + round(PA_R/100)
                                CS_L = int(round(SB_L/4))
                                CS_R = int(round(SB_R/4))

                                for num in range(1, on_base):
                                    attempt = on_base-num
                                    walks = num
                                    average = attempt/pa
                                    if(average<avg):
                                        hits = attempt
                                        break

                                BB_L = int(math.floor(walks*5/18))
                                BB_R = walks-BB_L
                                AB_L = PA_L-BB_L
                                AB_R = PA_R-BB_R
                                SO_L = int(math.floor((pa-on_base)*(.24*multiplier+(.25-avg))))
                                SO_R = int(math.floor((pa-on_base)*(.24/multiplier+(.25-avg))))

                                H_L = int(math.floor(hits/(4*multiplier*multiplier)))
                                AVG_L = H_L/AB_L
                                singles_L = int(round(H_L*.58/multiplier))
                                doubles_L = int(round(H_L*.22/multiplier))
                                triples_L = int(round(H_L/(70*multiplier)))
                                HR_L = H_L-singles_L-doubles_L-triples_L
                                temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L

                                H_R = hits-H_L
                                AVG_R = H_R/AB_R
                                singles_R = int(round(H_R*.58*multiplier))
                                doubles_R = int(round(H_R*.2*multiplier))
                                triples_R = int(round(H_R*multiplier/70))
                                HR_R = H_R-singles_R-doubles_R-triples_R
                                temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R

                                if(temp_slg_L>slg/multiplier):
                                    while(temp_slg_L>slg/multiplier and doubles_L>1):
                                        doubles_L-=1
                                        singles_L+=1
                                        temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L
                                elif(temp_slg_L<slg/multiplier):
                                    while(temp_slg_L<slg/multiplier and singles_L>1):
                                        doubles_L+=1
                                        singles_L-=1
                                        temp_slg_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/AB_L

                                if(temp_slg_R>slg*multiplier):
                                    while(temp_slg_R>slg/multiplier and doubles_R>1):
                                        doubles_R-=1
                                        singles_R+=1
                                        temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R
                                elif(temp_slg_R<slg*multiplier):
                                    while(temp_slg_R<slg*multiplier and singles_R>1):
                                        doubles_R+=1
                                        singles_R-=1
                                        temp_slg_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/AB_R

                                stats = pd.Series({'Season': 2019, 'Name': row['Name'], 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Bats': bats, 'G_L': G_L, 'PA_L': PA_L, 'AB_L': AB_L, 'H_L': H_L, '1B_L':singles_L, '2B_L': doubles_L, 
                                                   '3B_L': triples_L, 'HR_L': HR_L, 'R_L': R_L, 'RBI_L': RBI_L, 'BB_L': BB_L, 'IBB_L': 0, 
                                                   'SO_L': SO_L, 'HBP_L': HBP_L, 'SF_L': SF_L, 'SH_L': SH_L, 'GDP_L': GDP_L, 'SB_L': SB_L, 
                                                   'CS_L': CS_L, 'AVG_L': AVG_L, 'G_R': G_R, 'PA_R': PA_R, 'AB_R': AB_R, 'H_R': H_R, '1B_R': singles_R, 
                                                   '2B_R': doubles_R, '3B_R': triples_R,'HR_R': HR_R, 'R_R': R_R, 'RBI_R': RBI_R, 'BB_R': BB_R, 'IBB_R': 0, 
                                                   'SO_R': SO_R, 'HBP_R': HBP_R,'SF_R': SF_R, 'SH_R': SH_R, 'GDP_R': GDP_R, 'SB_R': SB_R, 'CS_R': CS_R, 
                                                   'AVG_R': AVG_R, 'BsR': float(player_values['BsR']), 'Fld': float(player_values['Fld']), pos: int(row['PA'])}) 

                            if(batter_df.empty):
                                batter_df = pd.DataFrame(columns=col_headers)

                            batter_df = batter_df.append(stats, ignore_index=True)
                            
            elif(pos in ['SP', 'RP']):
                for index, row in df.iterrows():
                    if(row['ID'] in pitcher_stats['playerId'].values.tolist()):
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            pitcher_df.loc[pitcher_df['playerId']==row['ID'], pos] = float(row['IP'])
                        else:
                            stats = pitcher_stats.loc[pitcher_stats['playerId']==row['ID']]
                            stats[pos] = float(row['IP'])
                            col_headers = stats.columns

                            if(pitcher_df.empty):
                                pitcher_df = pd.DataFrame(columns=col_headers)

                            pitcher_df = pitcher_df.append(stats, ignore_index=True)
                    else:
                        if(not pitcher_df.empty and row['ID'] in pitcher_df['playerId'].values.tolist()):
                            pitcher_df.loc[pitcher_df['playerId']==row['ID'], pos] = float(row['IP'])
                        else:
                            player_df = positional_values_dict['ALL Pitchers']
                            throws = id_arms_dict[row['ID']]

                            new_player_id.update({row['Name']: row['ID']})
                            player_values = player_df.loc[player_df['ID']==row['ID']]

                            if(float(player_values['IP'])<20):
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'], 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Throws': throws, 'IP_L': 30, 'TBF_L': 127, 'ERA_L': 5.40, 'H_L': 30, '1B_L': 20, 
                                                   '2B_L': 5, '3B_L': 1, 'R_L': 18, 'ER_L': 18, 'HR_L': 4, 'BB_L': 17, 'IBB_L': 0,  
                                                   'HBP_L': 3, 'SO_L': 50, 'AVG_L': .250, 'OBP_L': .385, 'SLG_L': .544, 
                                                   'wOBA_L': .370, 'IP_R': 30, 'TBF_R': 127, 'ERA_R': 5.40, 'H_R': 30, 
                                                   '1B_R': 20, '2B_R': 5, '3B_R': 1, 'R_R': 18, 'ER_R': 18, 'HR_R': 4, 
                                                   'BB_R': 17, 'IBB_R': 0, 'HBP_R': 3, 'SO_R': 50, 'AVG_R': .250, 
                                                   'OBP_R': .385, 'SLG_R': .544, 'wOBA_R': .370, pos: int(float(row['IP']))})


                            else:
                                if(throws=='L'):
                                    multiplier = 1.05
                                elif(throws=='R'):
                                    multiplier = 1/1.05
                                else:
                                    multiplier = 1

                                ip = int(round(float(player_values['IP'])))
                                ip_L = ip*.4
                                ip_R = ip-ip_L
                                outs_L = int(round(ip_L*3))
                                outs_R = int(round(ip_R*3))
                                SO_L = int(float(player_values['K/9'])*ip_L*multiplier/9)
                                SO_R = int(float(player_values['K/9'])*ip_L/(multiplier*9))
                                ER_L = int(round(ip_L*float(player_values['ERA'])/(multiplier*9)))
                                ER_R = int(round(ip_L*float(player_values['ERA'])*multiplier/9))
                                R_L = int(round(ER_L*1.07))
                                R_R = int(round(ER_R*1.07))
                                ERA_L = ER_L*27/outs_L
                                ERA_R = ER_R*27/outs_R
                                BB_L = int(float(player_values['BB/9'])*ip_L/(multiplier*9))
                                BB_R = int(float(player_values['BB/9'])*ip_R*multiplier/9)
                                temp_whip_L = ERA_L/(10*multiplier) + .8
                                temp_whip_R = ERA_R*multiplier/10 + .8
                                temp_wh_L = int(round(temp_whip_L*ip_L))
                                temp_wh_R = int(round(temp_whip_R*ip_R))
                                HR_L =  int(float(player_values['HR/9'])*ip_L/(multiplier*9))
                                HR_R = int(float(player_values['HR/9'])*ip_R*multiplier/9)
                                triples_L = int(round(outs_L*.005))
                                triples_R = int(round(outs_R*.005))
                                doubles_L = int(round(outs_L/(17*multiplier)))
                                doubles_R = int(round(outs_R*multiplier/17))            
                                singles_L = temp_wh_L-HR_L-triples_L-doubles_L-BB_L
                                singles_R = temp_wh_R-HR_R-triples_R-doubles_R-BB_R
                                TBF_L = int(round(.98*(outs_L+temp_wh_L)))
                                TBF_R = int(round(.98*(outs_R+temp_wh_R)))
                                H_L = singles_L+doubles_L+triples_L+HR_L
                                H_R = singles_R+doubles_R+triples_R+HR_R
                                HBP_L = int(round(TBF_L*.01))
                                HBP_R = int(round(TBF_R*.01))
                                IBB_L = int(round(BB_L/200))
                                IBB_R = int(round(BB_R/200))
                                AVG_L = H_L/(TBF_L-HBP_L-BB_L)
                                AVG_R = H_R/(TBF_R-HBP_R-BB_R)
                                OBP_L = (H_L+BB_L+HBP_L)/TBF_L
                                OBP_R = (H_R+BB_R+HBP_R)/TBF_R
                                SLG_L = (singles_L+doubles_L*2+triples_L*3+HR_L*4)/(TBF_L-HBP_L-BB_L)
                                SLG_R = (singles_R+doubles_R*2+triples_R*3+HR_R*4)/(TBF_R-HBP_R-BB_R)
                                woba_L = (.69*IBB_L+.719*HBP_L+.87*singles_L+1.217*doubles_L+1.529*triples_L+1.94*HR_L)/(TBF_L-IBB_L+HBP_L+.6*TBF_L)
                                woba_R = (.69*IBB_R+.719*HBP_R+.87*singles_R+1.217*doubles_R+1.529*triples_R+1.94*HR_R)/(TBF_R-IBB_R+HBP_R+.6*TBF_R)
                                
                                stats = pd.Series({'Season': 2019, 'Name': row['Name'], 'playerId': row['ID'], 'Team': nickname_to_city_dict[team_name.lower()], 
                                                   'Throws': throws, 'IP_L': ip_L, 'TBF_L': TBF_L, 'ERA_L': ERA_L, 'H_L': H_L, '1B_L': singles_L, 
                                                   '2B_L': doubles_L, '3B_L': triples_L, 'R_L': R_L, 'ER_L': ER_L, 'HR_L': HR_L, 'BB_L': BB_L, 'IBB_L': IBB_L,  
                                                   'HBP_L': HBP_L, 'SO_L': SO_L, 'AVG_L': AVG_L, 'OBP_L': OBP_L, 'SLG_L': SLG_L, 
                                                   'wOBA_L': woba_L, 'IP_R': ip_R, 'TBF_R': TBF_R, 'ERA_R': ERA_R, 'H_R': H_R, 
                                                   '1B_R': singles_R, '2B_R': doubles_R, '3B_R': triples_R, 'R_R': R_R, 'ER_R': ER_R, 'HR_R': HR_R, 
                                                   'BB_R': 17, 'IBB_R': 0, 'HBP_R': 3, 'SO_R': 50, 'AVG_R': .250, 
                                                   'OBP_R': OBP_R, 'SLG_R': SLG_R, 'wOBA_R': woba_R, pos: int(float(row['IP']))})
                   
                            if(pitcher_df.empty):
                                pitcher_df = pd.DataFrame(columns=col_headers)

                            pitcher_df = pitcher_df.append(stats, ignore_index=True)
                
                
        batter_df = batter_df.append(batter_stats.loc[batter_stats['Name']=='Avg_Totals'])
        batter_df.fillna(0, inplace=True)
        
        pitcher_df = pitcher_df.append(pitcher_stats.loc[pitcher_stats['Name']=='Avg_Totals'])
        pitcher_df.fillna(0, inplace=True)

        for pos in ['C', '1B', '2B', 'SS', '3B', 'LF', 'CF', 'RF', 'DH']:
            batter_df.loc[batter_df.Name=='Avg_Totals', pos] = batter_df.sum()[pos]
            
        batter_df.loc[batter_df.Name=='Avg_Totals', 'BsR'] = batter_df.sum()['BsR']
        batter_df.loc[batter_df.Name=='Avg_Totals', 'Fld'] = batter_df.sum()['Fld']
            
        for pos in ['SP', 'RP']:
            pitcher_df.loc[pitcher_df.Name=='Avg_Totals', pos] = pitcher_df.sum()[pos]    

        city = nickname_to_city_dict[team_name.lower()]
        
        team_dict.update({city: Team(city, league[city], batter_df, pitcher_df, starters_dict[city])})
        
    return team_dict

In [ ]:
batter_stats_df = reading_data('Splits_Leaderboard_Data_Hitters.csv')
pitcher_stats_df = reading_data('Splits_Leaderboard_Data_Pitchers.csv')
batter_stats_df, pitcher_stats_df = column_type(batter_stats_df, pitcher_stats_df)
schedule = reading_data('2020_mlb_schedule.csv')
team_dict = create_teams(batter_stats_df, pitcher_stats_df)